# Introduction

## Contexte et objectifs

Ce projet analyse un dataset Spotify contenant des caractéristiques audio de milliers de chansons provenant de différents genres musicaux. L'objectif principal est d'explorer la structure des données musicales à travers diverses techniques d'analyse multivariée et de réduction de dimension.

## Dataset

Le dataset comprend des variables quantitatives (danceability, energy, valence, tempo, etc.) et qualitatives (genre, artiste, tonalité, mode) permettant une analyse complète des profils musicaux. Après prétraitement, nous disposons de données sur les caractéristiques audio objectives et les métadonnées descriptives des morceaux.

## Approche méthodologique

Cette analyse combine plusieurs techniques complémentaires :
- **Analyse en Composantes Principales (ACP)** pour explorer les relations entre variables quantitatives et réduire la dimensionnalité
- **Analyse des Correspondances Multiples (MCA)** pour étudier les associations entre variables qualitatives (genres, tonalités, modes)
- **Analyse Factorielle Multiple (MFA)** pour intégrer simultanément données quantitatives et qualitatives dans une analyse unifiée
- **Techniques de réduction non-linéaire** (MDS, t-SNE) pour visualiser la structure complexe des données musicales
- **Factorisation Matricielle Non-négative (NMF)** pour identifier des profils musicaux latents et développer un système de recommandation
- **Algorithmes de clustering** pour segmenter les morceaux en groupes homogènes selon leurs caractéristiques audio

L'objectif est de comprendre comment les caractéristiques musicales s'organisent, identifier des patterns et profils musicaux distincts, et développer des applications pratiques comme un système de recommandation personnalisé basé sur les profils latents découverts.

# Data pre-processing

## Préparation et nettoyage des données

Le préprocessing est une étape cruciale qui conditionne la qualité des analyses statistiques ultérieures. Pour ce dataset Spotify, plusieurs transformations sont nécessaires :

### Objectifs du préprocessing

- **Élimination des variables non pertinentes** : Suppression des identifiants techniques (`track_id`, `album_id`, `playlist_id`) qui n'apportent pas d'information analytique
- **Gestion des valeurs manquantes** : Identification et traitement des données incomplètes pour éviter les biais dans les analyses
- **Standardisation des types de données** : Conversion des variables catégorielles et temporelles dans les formats appropriés
- **Déduplication intelligente** : Conservation des versions les plus populaires des morceaux dupliqués
- **Harmonisation des unités** : Conversion de la durée en secondes pour une meilleure interprétabilité

### Impact sur les analyses multivariées

Un preprocessing rigoureux garantit :
- Des résultats d'ACP non biaisés par des échelles de variables hétérogènes
- Une MCA cohérente avec des modalités catégorielles bien définies
- Des algorithmes de clustering et de réduction de dimension plus performants
- Une meilleure généralisation des modèles de recommandation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import numpy as np

In [ ]:
data = pd.read_csv('data/spotify_songs.csv')

In [ ]:
display(data)
display(data.describe().T.style.background_gradient(cmap='YlGnBu'))

In [ ]:
# check for missing values
missing_values = data.isnull().sum()
display(missing_values)

# display the lines with missing values
missing_data = data[data.isnull().any(axis=1)]
display(missing_data)

In [ ]:
# Remove id columns
data = data.drop(columns=['track_id', 'track_album_id', "playlist_id"], axis=1)
# drop the missing values
data = data.dropna()
display(data)

In [ ]:
# transform track_album_release_date into datetime
data['track_album_release_date'] = pd.to_datetime(data['track_album_release_date'], format='mixed')

# transform categorical columns into categorical data type
categorical_cols = ['playlist_genre', 'playlist_subgenre', 'track_artist', 'playlist_name', 'track_album_name']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# transform the duration_ms into minutes
data['duration_s'] = data['duration_ms'] / 1000
data.drop(columns=['duration_ms'], inplace=True)

# For numeric columns that represent discrete values (like key and mode), convert to categorical
key_mapping = {
    0: 'C', 1: 'C♯/D♭', 2: 'D', 3: 'D♯/E♭', 4: 'E', 5: 'F',
    6: 'F♯/G♭', 7: 'G', 8: 'G♯/A♭', 9: 'A', 10: 'A♯/B♭', 11: 'B'
}
data['key'] = data['key'].map(key_mapping).astype('category')

data['mode'] = data['mode'].map({0: 'Minor', 1: 'Major'}).astype('category')



In [ ]:
# check for duplicates
duplicates = data.duplicated().sum()
display(duplicates)

In [ ]:
data.info()

In [ ]:
num = data.select_dtypes(include='number')
cat = data.select_dtypes(include='category')
print('Data quantitative :', round(100*num.shape[1]/data.shape[1], 2), '%')
print('Data qualitative :', round(100*cat.shape[1]/data.shape[1], 2), '%')

In [ ]:
# copier data dans data_songs
data_songs = data.copy()

# observer les données dupliquées sur toutes les colonnes
duplicates = data_songs.duplicated().sum()
print(f'Number of duplicates: {duplicates}')

# Nombre de valeurs dupliquées avant suppression
initial_duplicates = data_songs.duplicated(subset=['track_artist', 'track_name']).sum()
print(f'Initial number of duplicates: {initial_duplicates}')

# Supprimer les colonnes 'playlist'
data_songs = data_songs.drop(columns=['playlist_genre', 'playlist_subgenre', 'playlist_name'])

# Regarder le nombre de doublons après suppression
duplicates = data_songs.duplicated().sum()
print(f'Number of duplicates - après suppression des playlists : {duplicates}')

# Supprimer les doublons
data_songs = data_songs.drop_duplicates()

# regarder le nombre de doublons en fonction de 'track_artist' et 'track_name'
duplicates = data_songs.duplicated(subset=['track_artist', 'track_name']).sum()
print(f'Number of duplicates - en se basant sur le nom d artiste de la track et : {duplicates}')

# garder dans data_songs les lignes dupliquées ayant la popularité la plus élevée
data_songs = data_songs.sort_values('track_popularity', ascending=False).drop_duplicates(subset=['track_artist', 'track_name'])
# regarder le nombre de doublons en fonction de 'track_artist' et 'track_name'
duplicates = data_songs.duplicated().sum()
print(f'Number of duplicates - après suppression des supposés doublons : {duplicates}')

In [ ]:
# Trier data_songs par l'index 'idx'
data_songs.sort_index(inplace=True)

display(data_songs[-5:-1])

Dans ce pré-processing de nos données, nous avons fait différents choix pour traiter nos données qui sont les suivantes :
* Nous avons supprimé les lignes du jeu avec des valeurs manquantes (nom d'artiste, de morceau...), et nous avons constaté qu'uniquement **5 lignes** avaient des données non spécifiées. Nous avons donc fait le choix de les supprimer en supposant que cela n'aurait pas d'impact, étant donné la taille du jeu de données initial.
* Nous avons également supprimé les colonnes qui représentaient les identifiants des artistes, playlists et morceaux car nous avons décidé de ne pas nous baser dessus dans notre étude car c'est une suite de lettres et de chiffres. 
* Nous avons effectué quelques modifications sur les variables, en transformant certaines variables numériques en variables catégoriques et certains format d'affichage. 

Nous avons finalement souhaité étudier notre jeu de données initial en considérant deux jeux. Ce sont les suivants :
* Dans certains cas nous utilisons le **jeu de données initialement fourni** (avec les modification effectuées dessus).
* Dans d'autres cas nous utilisons un **jeu de données qui ne se base pas sur les playlists** auxquels les morceaux sont associés. Cela a permis de ne pas considérer les morceaux présents dans plusieurs playlists (identiques ou différentes) afin de pouvoir considérer uniquement leurs caractéristiques. De plus, afin de supprimer les éventuels *doublons* présents, nous avons fait le choix de conserver le morceau ayant la plus haute popularité. En effet, certains artistes sortent des EP et ces mêmes EPs sont également présents dans des albums, mais n'ont pas la même popularité. Donc pour ne pas biaiser d'éventuelles analyses, ce choix a été fait. 

# Analyse exploratoire 

## Analyse univariée

In [ ]:
# Fonction pour tracer les variables en fonction du type d'affichage voulu
def plot_specific_variables(data, variable_plot_map):
    num_plots = len(variable_plot_map)
    fig, axes = plt.subplots(nrows=(num_plots + 1) // 2, ncols=4, figsize=(18, 6 * ((num_plots + 3) // 4)))#, constrained_layout=True)
    axes = axes.flatten()

    plot_index = 0

    for var, plot_type in variable_plot_map.items():
        if plot_type == 'histogram':
            sns.histplot(data[var], kde=True, ax=axes[plot_index])
            axes[plot_index].set_title(f'Distribution of {var}')
            axes[plot_index].set_xlabel(var)
            axes[plot_index].set_ylabel('Frequency')
        elif plot_type == 'countplot':
            sns.countplot(x=var, data=data, order=data[var].value_counts().index, ax=axes[plot_index])
            axes[plot_index].set_title(f'Count of {var}')
            axes[plot_index].set_xlabel(var)
            axes[plot_index].set_ylabel('Frequency')
        elif plot_type == 'boxplot':
            sns.boxplot(x=data[var], ax=axes[plot_index])
            axes[plot_index].set_title(f'Boxplot of {var}')
            axes[plot_index].set_xlabel(var)
            axes[plot_index].set_ylabel('Frequency')
        elif plot_type == 'kde':
            sns.kdeplot(data[var], ax=axes[plot_index])
            axes[plot_index].set_title(f'KDE of {var}')
            axes[plot_index].set_xlabel(var)
            axes[plot_index].set_ylabel('Density')
        else:
            raise ValueError(f"Unknown plot type: {plot_type}")

        plot_index += 1

    # Masquer les sous-graphiques non utilisés
    for i in range(plot_index, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
# Mappage des variables aux types de graphiques
variable_plot_map = {
    'track_popularity': 'histogram',
    'danceability': 'histogram',
    'energy': 'histogram',
    'loudness': 'histogram',
    'acousticness': 'histogram',
    'valence': 'histogram',
    'tempo': 'histogram',
    'duration_s': 'histogram'
}

plot_specific_variables(data_songs, variable_plot_map)

Nous pouvons constater que la majorité des caractéristiques représentées suivent une distribution qui semble normale.  

Concernant la distribution de la **popularité**, il y a une grande fréquence de morceaux dont la popularité est nulle. Cela s'explique par les critères de notations de *Spotify*, qui note la popularité d'un morceaux en fonction de ses écoutes, si elles ont étés nombreuses ou encore récentes.  

L'**acoustique** d'un morceau est noté entre 0.0 et 1.0. 
* 0.0 représente un morceau perçu comme non acoustique.
* 1.0 représente à l'inverse un morceau acoustique.  
  
Grâce à la  distribution, nous pouvons constater qu'une grande fréquence des morceaux présents dans ce jeu possèdent une acoustique nulle, cela peut déjà donner une première analyse sur le fait que le jeu de données est principalement composé de musiques de type *électronique* ou *synthétique*. 

Nous avons décidé ensuite de créer des intervalles pour certaines variables afin qu'elles soient plus facilement interprétables.

In [ ]:
bins_speechiness = [-float('inf'), 0.33, 0.66, float('inf')]
labels_speechiness = ['<0.33', '[0.33;0.66]', '>0.66']
data_songs['speechiness_interval'] = pd.cut(data_songs['speechiness'], bins=bins_speechiness, labels=labels_speechiness)

bins_instrumentalness = [-float('inf'), 0.5, float('inf')]
labels_instrumentalness = ['<=0.5', '>0.5']
data_songs['instrumentalness_interval'] = pd.cut(data_songs['instrumentalness'], bins=bins_instrumentalness, labels=labels_instrumentalness)

bins_liveness = [-float('inf'), 0.8, float('inf')]
labels_liveness = ['<=0.8', '>0.8']
data_songs['liveness_interval'] = pd.cut(data_songs['liveness'], bins=bins_liveness, labels=labels_liveness)

speechiness_counts = data_songs['speechiness_interval'].value_counts().sort_index()
instrumentalness_counts = data_songs['instrumentalness_interval'].value_counts().sort_index()
liveness_counts = data_songs['liveness_interval'].value_counts().sort_index()


# Créer une figure avec 3 sous-graphiques sur une même ligne
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Premier graphique : speechiness_interval
axes[0].bar(range(len(speechiness_counts)), speechiness_counts.values, color='skyblue')
axes[0].set_xticks(range(len(speechiness_counts)))
axes[0].set_xticklabels(speechiness_counts.index, rotation=45)
axes[0].set_title('Distribution des intervalles de paroles')
axes[0].set_xlabel('Intervalle de paroles')
axes[0].set_ylabel('Fréquence')

# Deuxième graphique : instrumentalness_interval
axes[1].bar(range(len(instrumentalness_counts)), instrumentalness_counts.values, color='lightcoral')
axes[1].set_xticks(range(len(instrumentalness_counts)))
axes[1].set_xticklabels(instrumentalness_counts.index, rotation=45)
axes[1].set_title('Distribution des intervalles d\'instrumentalité')
axes[1].set_xlabel('Intervalle d\'instrumentalité')
axes[1].set_ylabel('Fréquence')

# Troisième graphique : liveness_interval
axes[2].bar(range(len(liveness_counts)), liveness_counts.values, color='lightgreen')
axes[2].set_xticks(range(len(liveness_counts)))
axes[2].set_xticklabels(liveness_counts.index, rotation=45)
axes[2].set_title('Distribution des intervalles de présence live')
axes[2].set_xlabel('Intervalle de présence live')
axes[2].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

Pour les variables speechiness, instrumentalness et liveness, les intervalles dans lesquels les morceaux ont été classés, s'interprètent de la manière suivante : 
* **speechiness :** 
  * $<0.33$ : faible présence de paroles
  * $[0.33,0.66]$ : présence modérée de paroles
  * $>0.66$ : forte présence de paroles 
* **instrumentalness :** 
  * $\le 0.5$ : peu ou pas instrumental
  * $>0.5$ : très instrumental
* **liveness :** 
  * $\le 0.8$ : morceau probablement produit en studio
  * $>0.8$ : morceau probablement produit en live (concert, public)

D'après les distributions obtenues, la majorité des morceaux sont considérés comme possédant peu de paroles et qui n'ont pas été produits en live. Concernant l'instrumentalité des morceaux, nous pouvons difficilement conclure. 

**Années de sortie des albums**

Nous avons fait le choix de regrouper les albums par la décennie de leur sortie. Cela permet d'avoir une analyse plus pertinente que si l'on regardait par années de sortie. 

In [ ]:
# On extrait l'année de la date de sortie de l'album
data_songs['release_year'] = data_songs['track_album_release_date'].dt.year # type: ignore

# Création d'une nouvelle colonne pour les décennies
data_songs['decade'] = (data_songs['release_year'] // 10) * 10 # type: ignore

decade_counts = data_songs['decade'].value_counts().sort_index()

# Tracer la distribution des décennies
plt.figure(figsize=(12, 6))
sns.barplot(x=decade_counts.index, y=decade_counts.values, palette='viridis')
plt.title('Distribution des décennies de sortie des albums')
plt.xlabel('Décennie')
plt.ylabel('Nombre de chansons')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

La majorité des morceaux inclus dans le jeu de données sont des morceaux parus pendant les années 2010.

## Analyse multivariée

### Analyse des variables quantitatives

In [ ]:
# Sélection des colonnes numériques
numeric_columns = data_songs.select_dtypes(include=['int64', 'float64'])
correlation_matrix = numeric_columns.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

Les variables semblent très peu corrélées entre-elles pour la plupart, mais nous pouvons néanmoins constater les corrélations qui suivent.  
- **Corrélation positive :** 
    - *loudness* et *energy* : corrélation forte (0.69). Cela semble cohérent, on lie le dynamisme d'un morceau au bruit ressenti.
    - *valence* et *danceability* : légère corrélation (0.33). La valence mesure la positivité d'un morceau. Donc si celui-ci fait ressentir de la positivité, il y aura une tendance à donner l'envie de danser dessus.

- **Corrélation négative :**
    - *acousticness* et *energy* : corrélation négative modérée (-0.55). Plus une musique est perçue comme acoustique, moins elle est perçue comme énergique. Cela permet d'opposer des morceaux calmes à des morceaux dynamiques. 
    - *acousticness* et *loudness* : corrélation négative modérée (-0.38). Cela met en avant l'opposition entre un morceau bruyant et un morceau calme (acoustique).

Pour le reste on a des corrélations assez faibles, qu'elles soient positives ou négatives. Cela ne nous permet pas de mettre en avant d'autres liens entre les variables. 

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    
sns.scatterplot(x='loudness', y='energy', hue='track_popularity', data=data_songs, palette='viridis', alpha=0.7, ax=axes[0, 0])
axes[0, 0].set_title('Loudness vs Energy')
axes[0, 0].set_xlabel('Loudness')
axes[0, 0].set_ylabel('Energy')

sns.scatterplot(x='danceability', y='valence', hue='track_popularity', data=data_songs, palette='viridis', alpha=0.7, ax=axes[0, 1])
axes[0, 1].set_title('Danceability vs Valence')
axes[0, 1].set_xlabel('Danceability')
axes[0, 1].set_ylabel('Valence')

sns.scatterplot(x='acousticness', y='energy', hue='track_popularity', data=data_songs, palette='viridis', alpha=0.7, ax=axes[1, 0])
axes[1, 0].set_title('Acousticness vs Energy')
axes[1, 0].set_xlabel('Acousticness')
axes[1, 0].set_ylabel('Energy')

sns.scatterplot(x='acousticness', y='loudness', hue='track_popularity', data=data_songs, palette='viridis', alpha=0.7, ax=axes[1, 1])
axes[1, 1].set_title('Acousticness vs Loudness')
axes[1, 1].set_xlabel('Acousticness')
axes[1, 1].set_ylabel('Loudness')

for ax in axes.flatten():
    ax.get_legend().remove() 
    
handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, title='Popularity', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.suptitle('Scatterplots des corrélations entre les variables quantitatives (colorés par popularité)',
             fontsize=22)
plt.tight_layout(rect=[0, 0, 1, 0.96], w_pad=3)
plt.show()

Afin de vérifier ces corrélations, nous avons affiché les scatterplots les représentant.  
Les deux premiers graphiques (première ligne) représentent bien des **corrélations positives**, en effet à mesure qu'une des deux variables augmente, la seconde aussi.   
Pour les deux autres graphiques (seconde ligne), à l'inverse nous remarquons bien la **corrélation négative**. En effet lorsqu'une des deux variables évolue, la seconde diminue.   

Nous avons décidé d'effectuer l'affichage en fonction de la popularité, mais comme nous pouvous le voir, il y a des morceaux de différentes popularités qui sont placés aux mêmes endroits, nous pouvons donc penser que la popularité n'a aucune influence sur les variables représentées.

### Analyse des variables quantitatives et qualitatives 

**Étude de la popularité des morceaux par années**  
Comme cela a été présenté dans le pré-processing, nous étudions le jeu de données dans lequel nous avons extrait les morceaux uniques en gardant celui qui avait la popularité la plus haute, si ce même morceau était présent plusieurs fois.   
Cette popularité est calculée en fonction du nombre d'écoutes et si ces mêmes écoutes ont été récentes ou non. 

Nous allons ici étudier la date de sortie des morceaux en fonction de leur popularité.

In [ ]:
# Calcul de la moyenne de popularité par année
popularity_by_year = data_songs.groupby('release_year')['track_popularity'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x='release_year', y='track_popularity', data=popularity_by_year, marker='o')
plt.title('Popularité moyennes des morceaux par années')
plt.xlabel('Année')
plt.ylabel('Popularité moyenne')
plt.grid(True)
plt.show()


In [ ]:
popularity_by_decade = data_songs.groupby('decade')['track_popularity'].mean().reset_index()

plt.figure(figsize=(8, 5))
sns.lineplot(x='decade', y='track_popularity', data=popularity_by_decade, marker='o')
plt.title('Popularité moyenne des morceaux par décénnies')
plt.xlabel('décénnie')
plt.ylabel('Popularité moyenne')
plt.grid(True)
plt.show()

In [ ]:
# Nombre total de morceaux par décennie
count_by_decade = data_songs['decade'].value_counts().sort_index()
std_by_decade = data_songs.groupby('decade')['track_popularity'].std()

# Nombre de morceaux non notés (popularité = 0) par décennie
count_zero_popularity_by_decade = data_songs[data_songs['track_popularity'] == 0].groupby('decade').size()
proportion_zero_popularity = (count_zero_popularity_by_decade / count_by_decade * 100).fillna(0)

proportion_df = pd.DataFrame({
    'Nb total de morceaux': count_by_decade,
    '% de morceaux non notés': proportion_zero_popularity.round(2),
    '% de morceaux notés' : 100-proportion_zero_popularity.round(2),
    'Ecart-type': std_by_decade.round(2)
}).fillna(0)

print("\nProportion de morceaux non notés par décennie :")
print(proportion_df)

**Par années :**
- Sur le premier graphe on peut lire la popularité moyenne des morceaux par année. Cela reste difficilement interprétable car il y a beaucoup de variations. 
- Nous pouvons néanmoins constater qu'avant 1960 il y a une tendance de popularité haute, mais qu'après ça oscille entre 30 et 50/60. 
- La moyenne de popularité totale est de 40.45.  

Pour une interprétation plus globale nous avons regardé la popularité des morceaux en les regroupant par décennie de parution.

**Par décennies :**
  - On a une tendance de popularité très haute pour les musiques sorties entre 1950-1980. Cela peut s'expliquer notamment pour les musiques sorties entre 1950-60 : 
      - Peu de musiques sorties pendant ces années sont présentes dans le jeu de données.
      - Il y a une moins grande diversité dans les notes comme il y a moins de musiques. Donc si celles-ci sont très bien notées alors ces années auront nécessairement une moyenne de popularité élevée.
      - Si nous croisons ces résultats à l'écart-type associé qui est élevé, nous pouvons conclure que par le faible nombre de morceaux, cette décennie n'est pas représentative. Les décennies suivantes sont plus stables. 
  - Le minimum des moyennes de popularité par décennie est atteint pour les musiques des années 2000 :
      - Cela peut s'expliquer par le nombre de morceaux qui n'ont pas de note de popularité, avec presque 19% des morceaux qui ont une note de 0. 
      - Donc soit ces morceaux n'ont été que très peu écoutés, soit les morceaux n'ont pas connu de succès, augmentant l'écart-type par rapport aux années précédentes. 
  - À l'inverse, dès 2020, la dispersion est plus faible avec peu de morceaux non notés, mais un nombre de morceaux assez faible, donc les morceaux ont une popularité en moyenne assez similaire avec moins de morceaux à succès et non succès extrêmes. 


### Analyse multivariée en incluant le genre des playlists
**Analyse univariée sur les genres des playlists**

In [ ]:
# Affichage du nombre de musiques par genre/sous-genre
genre_counts = data['playlist_genre'].value_counts()
subgenre_counts = data['playlist_subgenre'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

genre_counts.plot(kind='bar', color='skyblue', ax=axes[0])
axes[0].set_title('Nombre de musiques par genre')
axes[0].set_xlabel('Genre')
axes[0].set_ylabel('Fréquence')
axes[0].tick_params(axis='x', rotation=45)

subgenre_counts.plot(kind='barh', color='lightcoral', ax=axes[1])
axes[1].set_title('Nombre de musiques par sous-genre')
axes[1].set_xlabel('Fréquence')
axes[1].set_ylabel('Sous-genre')

plt.tight_layout()
plt.show()

In [ ]:
pd.crosstab(data['playlist_genre'], data['playlist_subgenre'])

In [ ]:
# On compte le nombre de playlists différentes par genre
playlists_per_genre = data.groupby('playlist_genre', observed=False)['playlist_name'].nunique()

print("Nombre de playlists différentes par genre :")
print(playlists_per_genre)

Nous avons effectué une rapide analyse univariée sur les genres et les sous-genres des playlists, donc sur le jeu de donné fourni initialement.   

Nous pouvons remarquer que le nombre de playlists par genre est plutôt équilibré avec en moyenne 76 playlists par genres. Néanmoins, on peut constater que le genre **EDM** est majoritairement présent dans le jeu de données, et que le genre **rock** est celui qui est minoritaire. Le reste des genres se différencie uniquement de quelques centaines de morceaux.  
De plus, chaque sous-genre est associé à un unique genre, dont la répartition a été affichée. 

Nous allons maintenant croiser ces données avec nos autres variables afin de voir si nous pouvons trouver des relations ou non. 

In [ ]:
# Analyse des statistiques descriptives par genre
columns_to_analyze = ['danceability', 'energy', 'tempo', 'loudness']
stats_by_genre = data.groupby('playlist_genre', observed=False)[columns_to_analyze].agg(['mean', 'median', 'std', 'min', 'max'])

stats_by_genre.columns = ['_'.join(col).strip() for col in stats_by_genre.columns.values]

print("Statistiques descriptives par genre :")
display(stats_by_genre)

# Affichage des moyennes
mean_by_genre = stats_by_genre.filter(like='_mean')
mean_by_genre.columns = ['danceability_mean', 'energy_mean', 'tempo_mean', 'loudness_mean']
display(mean_by_genre)

In [ ]:
variables = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'track_popularity', 'duration_s'
]

n_cols = 3
n_rows = (len(variables) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows))
axes = axes.flatten()
handles, labels = None, None

for i, var in enumerate(variables):
    ax = axes[i]
    for genre in data['playlist_genre'].cat.categories:
        subset = data[data['playlist_genre'] == genre]
        sns.kdeplot(subset[var], ax=ax, label=genre, fill=False, linewidth=1.5)
    ax.set_title(f'Distribution de {var} par genre')
    ax.set_xlabel(var)
    ax.set_ylabel('Densité')
    ax.grid(True)
    if handles is None and ax.get_legend_handles_labels()[0]:
        handles, labels = ax.get_legend_handles_labels()
    if ax.get_legend() is not None:
        ax.get_legend().remove()
    

for j in range(len(variables), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()

if handles and labels:
    fig.legend(
        handles, labels,
        loc='lower left',
        bbox_to_anchor=(0.75, 0.08),
        ncol=2,  
        fontsize=16,
        frameon=False,
    )
    
plt.show()

Afin d'essayer de comprendre les spécificités des morceaux présents dans le jeu de données, nous avons analysé les différentes **distributions des variables quantitatives** selon le genre des playlists. L'objectif est d'identifier ici les différences pouvant caractériser les genres musicaux. Ces courbes de densité permettent de visualiser où se concentrent la majorité des morceaux.  

<u>*Analyse des distributions et des genres :*</u>

* **Danceability :**
  * Le genre **latin** et le **rap** se distinguent particulièrement avec des valeurs de danceability proche de 0.8, indiquant des morceaux très dansants pour ces genres. 
  * À l'inverse, le genre **rock** est plus dispersé, indiquant que certains morceaux sont très dansants, d'autres pas du tout.
  
* **EDM**
  * Ce genre semble se distinguer sur plusieurs points, en effet d'après les distributions, les morceaux du type EDM auront tendance à donner un ressenti **dynamique (energy)** (densité autour de 0.9), **bruyant** et se dinstingue par son **tempo** se trouvant autour de 130 BPM, valeur caractérisant généralement les morceaux d'EDM, techno...
  * Néanmoins, les musiques de ce genre musical ne semblent pas être ressenties comme positives ou joyeuses d'après leur valence.
  * Et on retrouve bien la corrélation négative avec l'**acousticness**.
  
* **Popularity :**
  * Si l'on ne prend en compte le pic à 0 (qui concerne les morceaux non notés), mais seulement au second pic, on peut remarquer :
    * **EDM** semble être le genre le moins populaire, mais la présence de la queue de distribution vers les valeurs élevées peut traduire la présence de morceaux à succès. 
    * Le **rap** possède une popularité plutôt moyenne par rapport aux autres genres, et ne semble pas avoir beaucoup de morceaux ayant connu un grand succès, mais l'inverse est aussi vrai, il ne semble pas y avoir beaucoup de morceaux à faible audience.
    * La **pop** et **latin** sont les genres les plus populaires.

* **Energy :**
  * L'**EDM** est le genre le plus dynamique. 
  * La distribution plus large pour les genres comme **r&b**, **rap** ou encore **latin** aura plutôt tendance à indiquer que ces genres regroupent à la fois des morceaux calmes et énergiques.

* **Valence :**
  * Le genre **latin** se distingue particulièrement par sa valence élevée, traduisant des musiques plutôt joyeuses.
  * La distribution globale de la valence est large (tous genres confondus) traduisant une grande diversité émotionnelle produite par les morceaux des différents genres. 

In [ ]:
n_cols = 3
n_rows = (len(variables) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows))
axes = axes.flatten()

handles, labels = None, None

for i, var in enumerate(variables):
    ax = axes[i]
    for genre in data['playlist_subgenre'].cat.categories:
        subset = data[data['playlist_subgenre'] == genre]
        sns.kdeplot(subset[var], ax=ax, label=genre, fill=False, linewidth=1.5)
    ax.set_title(f'Distribution de {var} par genre')
    ax.set_xlabel(var)
    ax.set_ylabel('Densité')
    ax.grid(True)
    if handles is None and ax.get_legend_handles_labels()[0]:
        handles, labels = ax.get_legend_handles_labels()
    if ax.get_legend() is not None:
        ax.get_legend().remove()

for j in range(len(variables), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()

if handles and labels:
    fig.legend(
        handles, labels,
        loc='lower left',
        bbox_to_anchor=(0.7, 0.03),
        ncol=2,  
        fontsize=12,
        frameon=False,
    )

plt.show()

Nous avons effectué le même affichage pour les sous-genres, qui sont un petit moins lisibles. Néanmoins, pour les analyses que nous avons fait précédemment, nous retrouvons les mêmes principales caractéristiques pour les sous-genres associés. 

# Réduction de dimension
Dans cette partie, nous allons effectuer une analyse en composantes principales (ACP) sur les données prétraitées. L'ACP est une technique de réduction de dimension qui permet de projeter les données d'origine dans un espace de dimension inférieure.
Nous avons gardé 20 variables et nous allons étudier s'il est possible de réduire la dimensionnalité de ces données tout en préservant un maximum d'information.

Ici on ne sélectionne que les variables quantitatives, en y ajoutant une variable qualitative (`playlist_genre`) pour voir si elle a un impact sur la projection des données. On garde au final 11 variables quantitatives et 1 variable qualitative.

On décide de normaliser les données pour que chaque variable ait une moyenne de 0 et un écart-type de 1. Cela est important car l'ACP est sensible à l'échelle des variables. On utilise la méthode `StandardScaler` de `sklearn` pour normaliser les données.

In [ ]:
# Select only the quantitative columns

qualisup = 'playlist_genre'

data_quanti = data.select_dtypes(include=['int64', 'float64'])
data_quanti[qualisup] = data[qualisup]
data_quanti = data_quanti.set_index(qualisup)

data_quanti.info()

In [ ]:
data_quanti.head()

## Analyse en Composantes Principales (ACP)

In [ ]:
from sklearn.decomposition import PCA
import prince 
from sklearn.preprocessing import StandardScaler

normalize_bool = True

if normalize_bool:
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_quanti)
else:
    data_scaled = data_quanti.values

pca = PCA(
    n_components=10,  # Number of components to keep
    random_state=1  # For reproducibility
)

projected = pca.fit_transform(data_scaled)

explained_variance = pca.explained_variance_ratio_

eig = pd.DataFrame(
    {
        "Dimension" : [f"PC{i+1}" for i in range(len(explained_variance))],
        "Variance" : np.round(pca.explained_variance_, 2),
        "% explained variance" : np.round(explained_variance*100, 1),
        "% cumulative variance" : np.round(np.cumsum(explained_variance)*100, 1)
    }
)
eig

In [ ]:
fig = plt.figure(figsize=(22,8))
ax = fig.add_subplot(1,2,1)
ax.bar(range(10), pca.explained_variance_ratio_[:10]*100, align='center',
        color='#F69F1D', ecolor='black')
ax.set_xticks(range(10))
ax.set_ylabel("Variance")
ax.set_title("", fontsize=35)
ax.set_title(u"Pourcentage de variance expliqué", fontsize=20)

ax = fig.add_subplot(1,2,2)
ax.plot(np.cumsum(pca.explained_variance_ratio_), color='#F69F1D', marker='o')
ax.hlines(0.80, 0, 10, colors='grey', linestyles='dashed', alpha=0.5)
ax.set_title(u'Pourcentage de variance expliqué cumulé', fontsize=20)

fig.suptitle(u"Résultat ACP", fontsize=25)
plt.show()

**Interprétation :** 

L’analyse de la variance expliquée montre que les **7 premières composantes principales** permettent de **représenter 80,1 % de la variance totale** du jeu de données. Cela signifie que l’essentiel de l’information contenue dans les **11 variables numériques initiales** (comme *danceability*, *energy*, *speechiness*, *tempo*, etc.) peut être résumé avec seulement 7 dimensions, ce qui représente une **réduction significative de la complexité** du dataset tout en conservant une bonne qualité descriptive.

Dans cette optique de réduction de dimension, il serait **pertinent de conserver ces 7 composantes principales** pour la suite des analyses (clustering, visualisation, classification), car elles capturent la structure principale des données tout en éliminant le "bruit".

Dans l’analyse factorielle, nous avons choisi d’**interpréter les trois premières composantes principales**, qui à elles seules expliquent 45 % de la variance totale. Elles offrent un bon compromis entre lisibilité et pertinence pour une visualisation ou une première analyse des relations entre les variables et les genres musicaux (*playlist_genre*).

In [ ]:
fig = plt.figure(figsize=(12,6))
box=plt.boxplot(projected[:,0:3],whis=100)
plt.title(u"Distribution des premières composantes", fontsize=20)
plt.show()

**Interprétation :**
- Le graphique ci-dessus montre la distribution des premières composantes principales. 

In [ ]:
pca_prince = prince.PCA(
    n_components=3,
    n_iter=3,
    rescale_with_mean=normalize_bool,
    rescale_with_std=normalize_bool,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=1
)
pca_prince = pca_prince.fit(
    data_quanti,
    sample_weight=None,
    column_weight=None,
    supplementary_columns=None
)

# Create a correlation matrix of the PCA compnenets
correlations = pca_prince.column_correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='RdBu', center=0)
plt.title('Corrélations entre les variables et les composantes principales')
plt.show()


In [ ]:
# Récupération des coordonnées des variables (correlations avec les composantes principales)
coords = pca_prince.column_correlations

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Boucle pour créer les trois graphiques
for idx, (ax, (x_comp, y_comp)) in enumerate(zip(axes, [(0, 1), (1, 2), (0, 2)])):
    ax.grid(False)
    for i in range(coords.shape[0]):
        ax.arrow(0, 0, coords.iloc[i, x_comp], coords.iloc[i, y_comp], 
                 color='black', alpha=0.7, head_width=0.02, head_length=0.03)
        ax.text(coords.iloc[i, x_comp] * 1.1, coords.iloc[i, y_comp] * 1.1, 
                coords.index[i], color='black', ha='center', va='center')

    # Ajout du cercle de rayon 1
    ax.add_artist(plt.Circle((0, 0), radius=1, color='cornflowerblue', fill=False))

    # Ajout des axes
    ax.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    ax.axvline(0, color='gray', linestyle='--', linewidth=0.5)

    # Paramètres du graphique
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel(f'Composante principale {x_comp + 1}')
    ax.set_ylabel(f'Composante principale {y_comp + 1}')
    ax.set_title(f'Projection des variables: CP{x_comp + 1} vs CP{y_comp + 1}')

plt.tight_layout()
plt.show()

La table des correlations et les trois graphiques ci-dessus représentent les projections des features sur les trois premières composantes principales nous donne des informations sur la structure des données réduites.

- **Composante principale 1** : 
    - Les variables `energy (-0.91)`, `loudness (-0.80)` et `acousticness (+0.72)` sont linéairement corrélées avec la première composante principale, en soulignant que `energy` et `loudness` sont inversément corrélées avec `acousticness`. Cela indique que la CP1 oppose les morceaux **énergiques, forts en volume et peu acoustiques** (ex : rock, électro) aux morceaux **calmes, acoustiques et peu énergétiques** (ex : folk, classique).
- **Composante principale 2** : Sur le graphique de gauche, on remarque une opposition des variables `instrumentalness (+0.45)`, `duration_s (+0.38)` contre `danceability (-0.68)`, `valence (-0.62)`, `track_popularity (-0.37)`, `speechiness (-0.39)`. Cette composante principale oppose deux profils de morceaux :
    - D’un côté, les morceaux **instrumentaux, longs et peu populaires** (forte contribution de `instrumentalness` et `duration_s`), souvent associés à des genres comme le classique ou le jazz.
    - De l’autre, les chansons **courtes, dansantes, joyeuses et populaires** (forte contribution de `danceability`, `valence` et `track_popularity`), typiques de la pop ou de la musique de club. 
    - Enfin, cette opposition suggère que les morceaux avec des paroles marquées (`speechiness`) et une structure rythmique engageante (`danceability`) sont plus susceptibles de générer de la popularité.
- **Composantes principales 2 et 3** : Sur le graphique au centre, on peut extraire plusieurs informations :
    - Les morceaux dansants et joyeux (haute valence) s'opposent dans une moindre mesure aux morceaux de faible tempo. De plus, ces morceaux semblent avoir peu de versions live.
    - On observe aussi que les morceaux instrumentaux et longs ont généralement une faible popularité, tandis que les morceaux courts et peu instrumentaux sont souvent plus populaires, ce qui est typique de la musique pop, qui est souvent plus accessible et commerciale.
- **Composante principale 3** : La troisième composante (CP3) révèle un paradoxe : elle regroupe des morceaux à fort potentiel dansant (`danceability`) et mood positif (`valence`), mais qui restent peu populaires (`track_popularity`). Ces morceaux sont souvent instrumentaux (`instrumentalness`), longs (`duration_s`) et à tempo faible (`tempo`), ce qui les éloigne des standards des charts. Cette composante pourrait représenter des créations artistiques équilibrant danse et complexité, mais peinant à atteindre un large public.

Pour mieux comprendre à quoi correspond les type morceaux extraits par ces composantes principales, nous allons regarder les morceaux (individus) contribuant le plus à chacune des composantes principales.

In [ ]:
# display la mediane de notre dataset quantitatif à fins de comparaison
median_values = data_quanti.median()
median_values = median_values.to_frame(name='median')
median_values = median_values.reset_index()
median_values.columns = ['feature', 'median']
median_values['feature'] = median_values['feature'].astype('category')
median_values = median_values.set_index('feature')
median_values = median_values.sort_index()
median_values = median_values.T
median_values

#### Composante principale 1

Afin de mieux comprendre les profils musicaux mis en évidence par la première composante principale, nous allons examiner les morceaux qui y contribuent le plus fortement, positivement et négativement.

In [ ]:
# Calculate the contributions of individuals to the first principal component
contributions = projected[:, 0]

# Get the indices of the top 5 positive contributors
top_5_positive_indices = np.argsort(contributions)[-5:]

# Get the indices of the top 5 negative contributors
top_5_negative_indices = np.argsort(contributions)[:5]

# Extract the corresponding rows from the original dataset for positive contributions
top_5_positive_tracks = data_quanti.iloc[top_5_positive_indices].copy()
top_5_positive_tracks['track_name'] = data.iloc[top_5_positive_indices]['track_name'].values
top_5_positive_tracks['track_artist'] = data.iloc[top_5_positive_indices]['track_artist'].values
top_5_positive_tracks['contribution'] = contributions[top_5_positive_indices]
top_5_positive_tracks['playlist_subgenre'] = data.iloc[top_5_positive_indices]['playlist_subgenre'].values
# sort by contribution
top_5_positive_tracks = top_5_positive_tracks.sort_values(by='contribution', ascending=False)

# Extract the corresponding rows from the original dataset for negative contributions
top_5_negative_tracks = data_quanti.iloc[top_5_negative_indices].copy()
top_5_negative_tracks['track_name'] = data.iloc[top_5_negative_indices]['track_name'].values
top_5_negative_tracks['track_artist'] = data.iloc[top_5_negative_indices]['track_artist'].values
top_5_negative_tracks['contribution'] = contributions[top_5_negative_indices]
top_5_negative_tracks['playlist_subgenre'] = data.iloc[top_5_negative_indices]['playlist_subgenre'].values

# Display the top 5 positive and negative tracks with additional information
print("Top 5 Positive Contributions (PC1):")
display(top_5_positive_tracks)

print("Top 5 Negative Contributions (PC1) :")
display(top_5_negative_tracks)

Pour mieux cerner les types de morceaux représentés aux extrémités de la **première composante principale (PC1)**, nous avons identifié les individus (chansons) ayant les **contributions les plus élevées**, positives comme négatives.

- **Du côté des contributions positives**, on retrouve majoritairement des morceaux **rock, hard rock ou pop rock** très énergiques et puissants tels que *American Idiot* (Green Day), *Beauty Queen* (BLVK SWVN) ou *ATTENTION ATTENTION* (Shinedown). Ces morceaux sont caractérisés par une **énergie élevée**, une **forte intensité sonore (loudness)** et une **faible acoustique**, ce qui confirme bien la structure mise en évidence par la CP1. Notons aussi *This Is How We Do It* (Montell Jordan), un morceau R&B énergique, qui se distingue des autres par son genre mais partage les mêmes caractéristiques acoustiques.

- **À l’opposé**, les morceaux à contribution très négative sur PC1 sont des titres à **forte acoustique**, **peu énergiques** et **très faibles en loudness**. Il s'agit notamment de sons **ambiants, relaxants ou naturels**, comme *Peaceful Forest* ou *Tropical Rainforest at Dawn*, mais aussi de titres R&B ou indie très doux (*Small* de chloe moriondo). Ces morceaux incarnent l'autre extrémité de la CP1 : **des chansons calmes, acoustiques et à faible énergie**, souvent issues de sous-genres comme *tropical*, *indie poptimism* ou *new jack swing*.

Cette opposition renforce l’interprétation de la **CP1 comme un axe énergie / intensité sonore vs. calme / acoustique**, pertinent pour distinguer deux grandes familles de styles musicaux dans le dataset.

#### Composante principale 2

Pour approfondir l’interprétation de la **deuxième composante principale**, nous allons analyser les morceaux qui y contribuent le plus fortement — positivement comme négativement.

In [ ]:
# Calculate the contributions of individuals to the second principal component
contributions_pc2 = projected[:, 1]

# Get the indices of the top 5 positive contributors for PC2
top_5_positive_indices_pc2 = np.argsort(contributions_pc2)[-5:]

# Get the indices of the top 5 negative contributors for PC2
top_5_negative_indices_pc2 = np.argsort(contributions_pc2)[:5]

# Extract the corresponding rows from the original dataset for positive contributions (PC2)
top_5_positive_tracks_pc2 = data_quanti.iloc[top_5_positive_indices_pc2].copy()
top_5_positive_tracks_pc2['track_name'] = data.iloc[top_5_positive_indices_pc2]['track_name'].values
top_5_positive_tracks_pc2['track_artist'] = data.iloc[top_5_positive_indices_pc2]['track_artist'].values
top_5_positive_tracks_pc2['contribution'] = contributions_pc2[top_5_positive_indices_pc2]
top_5_positive_tracks_pc2['playlist_subgenre'] = data.iloc[top_5_positive_indices_pc2]['playlist_subgenre'].values
# Sort by contribution
top_5_positive_tracks_pc2 = top_5_positive_tracks_pc2.sort_values(by='contribution', ascending=False)

# Extract the corresponding rows from the original dataset for negative contributions (PC2)
top_5_negative_tracks_pc2 = data_quanti.iloc[top_5_negative_indices_pc2].copy()
top_5_negative_tracks_pc2['track_name'] = data.iloc[top_5_negative_indices_pc2]['track_name'].values
top_5_negative_tracks_pc2['track_artist'] = data.iloc[top_5_negative_indices_pc2]['track_artist'].values
top_5_negative_tracks_pc2['contribution'] = contributions_pc2[top_5_negative_indices_pc2]
top_5_negative_tracks_pc2['playlist_subgenre'] = data.iloc[top_5_negative_indices_pc2]['playlist_subgenre'].values

# Display the top 5 positive and negative tracks with additional information for PC2
print("Top 5 Positive Contributions (PC2):")
display(top_5_positive_tracks_pc2)

print("Top 5 Negative Contributions (PC2):")
display(top_5_negative_tracks_pc2)

**Côté contributions positives**, on retrouve des titres principalement **rap et latino**, tels que *Suge* de DaBaby ou *LAX* de B0nds. Ces morceaux sont :
- **courts**,
- **dansants** (haute `danceability`),
- avec une **valence élevée** (émotion positive),
- mais également avec un certain niveau de **speechiness**, notamment pour les titres rap.

Ces morceaux partagent donc des caractéristiques propres aux chansons **énergétiques, rythmées et populaires**, souvent taillées pour le streaming, avec des formats courts, accrocheurs et directs.

**À l’opposé**, les morceaux ayant une **forte contribution négative à PC2** sont très différents : on retrouve des **paysages sonores naturels, ambiants ou instrumentaux** comme *Rain Forest and Tropical Beach Sound*, *Caribbean Thunderstorm*, ou encore *Battlement*. Ces titres sont :
- **longs**,
- **instrumentaux** (forte `instrumentalness`),
- avec une **faible valence** et **peu de parole**,
- et souvent issus de sous-genres comme *tropical*, *album rock*, ou *ambient*.

Cela confirme l’interprétation initiale de la PC2 comme un **axe opposant la musique instrumentale, longue et contemplative** à une musique **populaire, dansante et rythmée**.


#### Composante principale 3

In [ ]:
# Calculate the contributions of individuals to the third principal component
contributions_pc3 = projected[:, 2]

# Get the indices of the top 5 positive contributors for PC3
top_5_positive_indices_pc3 = np.argsort(contributions_pc3)[-5:]

# Get the indices of the top 5 negative contributors for PC3
top_5_negative_indices_pc3 = np.argsort(contributions_pc3)[:5]

# Extract the corresponding rows from the original dataset for positive contributions (PC3)
top_5_positive_tracks_pc3 = data_quanti.iloc[top_5_positive_indices_pc3].copy()
top_5_positive_tracks_pc3['track_name'] = data.iloc[top_5_positive_indices_pc3]['track_name'].values
top_5_positive_tracks_pc3['track_artist'] = data.iloc[top_5_positive_indices_pc3]['track_artist'].values
top_5_positive_tracks_pc3['contribution'] = contributions_pc3[top_5_positive_indices_pc3]
top_5_positive_tracks_pc3['playlist_subgenre'] = data.iloc[top_5_positive_indices_pc3]['playlist_subgenre'].values
# Sort by contribution
top_5_positive_tracks_pc3 = top_5_positive_tracks_pc3.sort_values(by='contribution', ascending=False)

# Extract the corresponding rows from the original dataset for negative contributions (PC3)
top_5_negative_tracks_pc3 = data_quanti.iloc[top_5_negative_indices_pc3].copy()
top_5_negative_tracks_pc3['track_name'] = data.iloc[top_5_negative_indices_pc3]['track_name'].values
top_5_negative_tracks_pc3['track_artist'] = data.iloc[top_5_negative_indices_pc3]['track_artist'].values
top_5_negative_tracks_pc3['contribution'] = contributions_pc3[top_5_negative_indices_pc3]
top_5_negative_tracks_pc3['playlist_subgenre'] = data.iloc[top_5_negative_indices_pc3]['playlist_subgenre'].values

# Display the top 5 positive and negative tracks with additional information for PC3
print("Top 5 Positive Contributions (PC3):")
display(top_5_positive_tracks_pc3)

print("Top 5 Negative Contributions (PC3):")
display(top_5_negative_tracks_pc3)

La **troisième composante principale** met en lumière une tension plus subtile entre deux types de morceaux aux caractéristiques inattendues.

**Du côté des contributions positives**, on retrouve des titres de **pop et R&B calmes et acoustiques** comme *raindrops (an angel cried)* (Ariana Grande) ou *You Are The Reason* (Calum Scott). Ces chansons ont :
- une **acousticness élevée** (acapela, guitare acoustique, piano),
- un **tempo légèrement plus rapide que la médiane des morceaux**,
- une **faible énergie**, mais un **potentiel émotionnel fort** (`valence` variable).

Comme suggéré par la PC3, ces morceaux rencontrent un certain succès, illustrant un profil de chansons émotionnelles, accessibles et bien produites, souvent interprétées par des artistes grand public au style sobre et expressif.

**Les contributions négatives**, quant à elles, sont largement dominées par des morceaux **EDM ou latino instrumentaux**, comme *I Feel Love* ou *Chase*. Ces morceaux sont :
- **longs**,
- très **instrumentaux**,
- **énergiques** mais souvent **moins "accessibles" émotionnellement** (valence très élevée mais peu de paroles, structure répétitive).

Ils présentent également une popularité extrêmement faible, allant de 0 à 8. Ces productions s’adressent probablement à un public averti ou sont conçues pour des usages spécifiques (DJ sets, ambiances electro), ce qui les éloigne des standards de la musique grand public.

Ces observations confirment que la **PC3 oppose des créations acoustiques à forte charge émotionnelle** à des morceaux **instrumentaux, électroniques, longs**, aux dynamiques parfois complexes ou répétitives.

On remarque que le signe des contribution est inversé par rapport à la PCA réalisé en R uniquement pour cette dimension.

# MDS

Le **MDS (Multidimensional Scaling)** est une technique de réduction de dimension non linéaire qui permet de visualiser des données en préservant les distances entre les points. Dans notre cas, nous allons l'appliquer sur les données prétraitées pour obtenir une représentation en 2D des morceaux.

In [ ]:
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances

df_data_scaled = pd.DataFrame(data_scaled, columns=data_quanti.columns)
# set the same index as data_quanti
df_data_scaled.index = data_quanti.index

# Limit to 500 songs randomly selected to avoid memory crash
data_scaled_sample = df_data_scaled.sample(n=500, random_state=1)

# Calculate the Manhattan distance matrix
manhattan_dist_matrix = manhattan_distances(data_scaled_sample)
# Calculate the Euclidean distance matrix
euclidean_dist_matrix = euclidean_distances(data_scaled_sample)

# Perform MDS using Manhattan distance
mds_manhattan = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
mds_manhattan_results = mds_manhattan.fit_transform(manhattan_dist_matrix)
# Perform MDS using Euclidean distance
mds_euclidean = MDS(n_components=2, dissimilarity='precomputed', random_state=1)
mds_euclidean_results = mds_euclidean.fit_transform(euclidean_dist_matrix)


In [ ]:
# Plot the MDS results for Manhattan and Euclidean distance in one figure with two subplots and a shared legend

# Prepare color palette for playlist_genre
palette = sns.color_palette('Set2', n_colors=data_scaled_sample.index.nunique())
genre_labels = data_scaled_sample.index.astype(str).values

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharex=False, sharey=False)

# Manhattan distance subplot
sns.scatterplot(
    x=mds_manhattan_results[:, 0],
    y=mds_manhattan_results[:, 1],
    hue=data_scaled_sample.index,
    palette=palette,
    alpha=0.7,
    ax=axes[0],
    legend=False  # Hide legend for now
)
axes[0].set_title('MDS Manhattan', fontsize=16)
axes[0].set_xlabel('MDS Dimension 1', fontsize=12)
axes[0].set_ylabel('MDS Dimension 2', fontsize=12)
axes[0].grid()

# Euclidean distance subplot
scatter = sns.scatterplot(
    x=mds_euclidean_results[:, 0],
    y=mds_euclidean_results[:, 1],
    hue=data_scaled_sample.index,
    palette=palette,
    alpha=0.7,
    ax=axes[1],
    legend='brief'  # Show legend only on this subplot
)
axes[1].set_title('MDS Euclidean', fontsize=16)
axes[1].set_xlabel('MDS Dimension 1', fontsize=12)
axes[1].set_ylabel('MDS Dimension 2', fontsize=12)
axes[1].grid()

# Place the legend outside the plot
handles, labels = scatter.get_legend_handles_labels()
fig.legend(handles, labels, title=qualisup, bbox_to_anchor=(1.02, 0.5), loc='center left')

plt.tight_layout()
plt.show()

In [ ]:
# perform several MDS sns.scatterplot with different hues, for the 11 quantitative variables in data_scaled
quantitative_vars = data_scaled_sample.columns

# Create a list of colors for the scatterplots
colors = sns.color_palette("Set2", len(quantitative_vars), as_cmap=True)
# Create a figure with subplots
fig, axes = plt.subplots(4, 3, figsize=(20, 20))
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration
# Loop through each quantitative variable and create a scatterplot
for i, var in enumerate(quantitative_vars):
    sns.scatterplot(
        x=mds_euclidean_results[:, 0],
        y=mds_euclidean_results[:, 1],
        hue=data_scaled_sample[var],
        ax=axes[i]
    )
    axes[i].set_title(f'MDS with {var}', fontsize=16)
    axes[i].set_xlabel('MDS Dimension 1', fontsize=14)
    axes[i].set_ylabel('MDS Dimension 2', fontsize=14)
    axes[i].legend(title=var, bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[i].grid()

plt.tight_layout()
plt.show()


**Interprétation :**

L’analyse en **MDS (Multidimensional Scaling)** appliquée aux données quantitatives donne des résultats contrastés.

* Lorsqu’on projette les points selon une **distance euclidienne ou manhattan**, et qu’on colore les morceaux selon le **genre de playlist**, **aucune structure claire n’émerge** : les genres sont **entièrement mélangés**. Cela montre que, globalement, **les variables quantitatives ne permettent pas de séparer les genres musicaux** dans une représentation MDS. La MDS **ne capture donc pas d'information utile liée au genre musical global** dans cet espace.

* Cependant, certaines **variables quantitatives spécifiques sont bien séparées** dans l’espace MDS :

  * C’est le cas de **`energy`**, **`loudness`**, **`speechiness`**, **`acousticness`**, **`instrumentalness`**, et dans une moindre mesure **`valence`**.

    * Pour **`valence`**, bien que l’on observe deux groupes principaux (valence haute vs. basse), **certains morceaux très joyeux se retrouvent mélangés avec des morceaux tristes**, ce qui rend la séparation moins nette que pour les autres variables.
  * On observe des **relations fortes et interprétables** entre certaines variables :

    * Sur la **dimension 1 de la MDS**, **`energy` et `loudness` sont corrélées positivement**, et **`acousticness` leur est exactement opposée**. Cela signifie que **les morceaux énergétiques et forts sont peu acoustiques**, ce qui est **parfaitement cohérent avec l’intuition musicale**.
    * Sur la **dimension 2**, on retrouve une opposition logique entre **`instrumentalness` et `speechiness`** : **les morceaux très instrumentaux sont peu parlés**, ce qui reflète bien la réalité musicale (ex : classique vs. rap).

* En revanche, pour **`track_popularity`**, **`danceability`**, **`tempo`** et **`duration_s`**, **aucune séparation nette n’apparaît** dans l’espace MDS. Ces variables semblent **ne pas être linéairement séparables** par la MDS. On peut donc conclure que **la MDS ne permet pas d’identifier de structure claire** pour ces dimensions — possiblement en raison de leur **relation non linéaire** ou d’un **bruit trop important**.

---

**Lien avec l’analyse PCA** :

On retrouve ici des observations **cohérentes avec l’analyse des composantes principales**. Par exemple :

* La **CP1** opposait déjà **`energy` et `loudness`** à **`acousticness`**, exactement comme la **dimension 1 de la MDS**.
* La **CP2** révélait une tension entre **`instrumentalness`** et **`speechiness`**, que la **MDS dimension 2** reflète également.
* Enfin, comme en PCA, **`track_popularity`**, **`tempo`** et **`duration_s`** ne permettent pas de bien structurer l’espace : ces variables n’étaient déjà que faiblement corrélées avec les premières composantes principales, et la MDS confirme leur faible capacité à structurer les données dans un espace réduit.

---

Nous allons désormais réaliser un **t-SNE** (t-distributed Stochastic Neighbor Embedding) pour visualiser en 2D la proximité (ou la similarité) entre les chansons Spotify selon leurs caractéristiques audio (danceability, energy, loudness, etc.), pour voir **si des genres musicaux distincts émergent sous forme de groupes**. A l'inverse de la PCA et de la MDS, le **t-SNE est une méthode non linéaire** qui pourrait capturer des relations différentes entre les données que celles révélées par les méthodes linéaires.

# t-SNE


In [ ]:
from sklearn.manifold import TSNE

# Perform t-SNE
tsne = TSNE(n_components=2, random_state=1)
tsne_results = tsne.fit_transform(data_scaled_sample)

# plot the t-SNE results with sns
plt.figure(figsize=(6, 6))
sns.scatterplot(
    x=tsne_results[:, 0],
    y=tsne_results[:, 1],
    hue=data_scaled_sample.index,
    palette='Set2',
    alpha=0.7
)
plt.title('t-SNE', fontsize=20)
plt.xlabel('t-SNE Dimension 1', fontsize=16)
plt.ylabel('t-SNE Dimension 2', fontsize=16)
plt.legend(title=qualisup, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.show()

In [ ]:
# Ajout du t-SNE dans la même figure (3 sous-graphiques sur une ligne)
fig, axes = plt.subplots(1, 3, figsize=(22, 8), sharex=False, sharey=False)

# Manhattan distance subplot
sns.scatterplot(
    x=mds_manhattan_results[:, 0],
    y=mds_manhattan_results[:, 1],
    hue=data_scaled_sample.index,
    palette=palette,
    alpha=0.7,
    ax=axes[0],
    legend=False
)
axes[0].set_title('MDS Manhattan', fontsize=16)
axes[0].set_xlabel('MDS Dimension 1', fontsize=12)
axes[0].set_ylabel('MDS Dimension 2', fontsize=12)
axes[0].grid()

# Euclidean distance subplot
scatter = sns.scatterplot(
    x=mds_euclidean_results[:, 0],
    y=mds_euclidean_results[:, 1],
    hue=data_scaled_sample.index,
    palette=palette,
    alpha=0.7,
    ax=axes[1],
    legend=False
)
axes[1].set_title('MDS Euclidean', fontsize=16)
axes[1].set_xlabel('MDS Dimension 1', fontsize=12)
axes[1].set_ylabel('MDS Dimension 2', fontsize=12)
axes[1].grid()

# t-SNE subplot
sns.scatterplot(
    x=tsne_results[:, 0],
    y=tsne_results[:, 1],
    hue=data_scaled_sample.index,
    palette=palette,
    alpha=0.7,
    ax=axes[2],
    legend='brief'
)
axes[2].set_title('t-SNE', fontsize=16)
axes[2].set_xlabel('t-SNE Dimension 1', fontsize=12)
axes[2].set_ylabel('t-SNE Dimension 2', fontsize=12)
axes[2].grid()

# Place the legend outside the plot (from the t-SNE plot)
handles, labels = axes[2].get_legend_handles_labels()
fig.legend(handles, labels, title=qualisup, bbox_to_anchor=(1.02, 0.5), loc='center left')

plt.tight_layout()
plt.show()

In [ ]:
# perform t-SNE with different hues, for the 11 quantitative variables in data_scaled
# Create a list of colors for the scatterplots
colors = sns.color_palette("Set2", len(quantitative_vars), as_cmap=True)
# Create a figure with subplots
fig, axes = plt.subplots(4, 3, figsize=(20, 20))
axes = axes.flatten()  # Flatten the 2D array of axes for easy iteration
# Loop through each quantitative variable and create a scatterplot
for i, var in enumerate(quantitative_vars):
    sns.scatterplot(
        x=tsne_results[:, 0],
        y=tsne_results[:, 1],
        hue=data_scaled_sample[var],
        ax=axes[i]
    )
    axes[i].set_title(f't-SNE with {var}', fontsize=16)
    axes[i].set_xlabel('t-SNE Dimension 1', fontsize=14)
    axes[i].set_ylabel('t-SNE Dimension 2', fontsize=14)
    axes[i].legend(title=var, bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[i].grid()

plt.tight_layout()
plt.show()

**Interprétation :**

Malgré l’application du **t-SNE**, on observe **toujours aussi peu de séparation nette entre les genres musicaux** : les morceaux restent largement **mélangés indépendamment de leur genre**, ce qui confirme les limites des caractéristiques audio pour discriminer les styles musicaux au sens large.

Cependant, le **t-SNE révèle une structure beaucoup plus claire que la MDS**, avec l’émergence de **groupes de morceaux** qui semblent s’organiser selon leurs **propriétés audio internes**.

* Les variables **`energy`** et **`loudness`** sont **clairement séparées**, avec une **distribution similaire**, ce qui rejoint parfaitement les résultats observés en **PCA (CP1)** et en **MDS (dim 1)**. De même, **`acousticness`** est **opposée** à ces deux variables, avec un groupe bien distinct de **musiques très acoustiques**, caractérisées par **une faible énergie et un faible volume sonore**.

* Concernant **`instrumentalness`**, le t-SNE apporte une **information nouvelle importante** : on distingue **deux groupes de morceaux instrumentaux** très différents :

  * Un groupe de morceaux **instrumentaux, calmes, acoustiques et peu louds**.
  * Un autre groupe **instrumental mais très énergétique, fort en loudness**, donc probablement plus proche de l’expérimental, du post-rock ou de l’électro instrumental.
    Cette séparation fine n’apparaissait **ni en PCA, ni en MDS**, et montre que le **t-SNE capture mieux les structures non linéaires** des données.

* Pour **`valence`**, on observe **plusieurs sous-groupes assez distincts**, mais **difficiles à interpréter directement**. Toutefois, en croisant avec **`acousticness`**, on remarque que **les morceaux acoustiques et calmes ont généralement une valence très faible**, ce qui correspond à des ambiances tristes ou introspectives.

* En revanche, pour des variables comme **`track_popularity`**, **`danceability`**, **`tempo`** ou **`duration_s`**, la séparation reste **floue et peu informative**, ce qui suggère que ces caractéristiques sont **moins déterminantes** pour structurer l’espace audio selon t-SNE.

---

**Bilan sur les méthodes de réduction de dimension quantitatives :**

* La **PCA** fournit une **lecture linéaire et interprétable** des relations entre variables, et met en évidence des axes clairs comme l’opposition **énergie vs. acoustique** ou **instrumental vs. populaire**.
* La **MDS**, malgré son approche plus flexible sur les distances, **n’apporte pas d’information supplémentaire significative** par rapport à la PCA, sauf sur des dimensions très spécifiques.
* Le **t-SNE**, quant à lui, **révèle des regroupements non linéaires** que les autres méthodes ne captent pas, et permet de **mieux explorer des structures complexes**, notamment pour des variables comme **`instrumentalness`**, **`energy`**, et **`acousticness`**.

En conclusion, chaque méthode a ses forces : **la PCA pour l’interprétation globale des variables**, **le t-SNE pour l'exploration fine des regroupements**, et **la MDS reste plus limitée** dans ce cas d’étude.


# Analyse en Correspondances Multiples (MCA)

### Objectif

L'Analyse en Correspondances Multiples (MCA) va être utilisée afin de visualiser les associations entre les modalités des variables qualitatives et d'identifier des clusters ou des oppositions significatives.

## Transformation des variables quantitatives en catégories

Pour enrichir l'analyse des correspondances multiples (MCA), nous proposons de convertir les variables quantitatives en variables qualitatives ordinales. Cette transformation permet d'intégrer les caractéristiques audio numériques dans une analyse factorielle adaptée aux données catégorielles.

### Critères de segmentation

La catégorisation suit une logique **musicologique et perceptuelle**, en tenant compte des seuils naturels d'interprétation des caractéristiques audio :

**Variables binaires (seuil à 0.5)** : 
- `danceability`, `energy`, `valence` : Séparation entre niveaux faible/élevé selon la médiane théorique
- `instrumentalness` : Distinction claire entre morceaux vocaux et instrumentaux

**Variables à seuils spécifiques** :
- `speechiness` : Seuil à 0.3 pour différencier musique pure vs. contenu parlé (rap, podcast)
- `liveness` : Seuil à 0.8 pour capturer les vraies performances live
- `popularity` : Segmentation en tertiles (0-20, 20-75, 75-100) reflétant les distributions réelles de Spotify

**Variables temporelles** :
- `tempo` : Classification musicologique standard (lent <100, modéré 100-150, rapide >150 BPM)
- `duration` : Segmentation basée sur les formats musicaux (courts <2min, moyens 2-4min, longs >4min)
- `decade` : Regroupement par décennies avec fusion des années 1950s-1960s pour équilibrer les effectifs

Cette approche préserve la signification musicale des variables tout en créant des catégories équilibrées pour l'analyse MCA.

In [ ]:
# Création d'un DataFrame catégorique selon les règles demandées

df_cat_custom = pd.DataFrame(index=data.index)

# Popularity
df_cat_custom['popularity_cat'] = pd.cut(
    data['track_popularity'],
    bins=[-1, 20, 75, 100],
    labels=['peu populaire', 'popularité moyenne', 'très populaire']
)

# Speechiness
df_cat_custom['speechiness_cat'] = pd.cut(
    data['speechiness'],
    bins=[-0.01, 0.33, 1.0],
    labels=['peu de paroles', 'paroles dominantes']
)

# Danceability
df_cat_custom['danceability_cat'] = pd.cut(
    data['danceability'],
    bins=[-0.01, 0.5, 1.0],
    labels=['peu dansant', 'dansant']
)

# Energy
df_cat_custom['energy_cat'] = pd.cut(
    data['energy'],
    bins=[-0.01, 0.5, 1.0],
    labels=['peu énergique', 'énergique']
)

# Instrumentalness
df_cat_custom['instrumentalness_cat'] = pd.cut(
    data['instrumentalness'],
    bins=[-0.01, 0.5, 1.0],
    labels=['peu instrumentale', 'instrumentale']
)

# Liveness
df_cat_custom['liveness_cat'] = pd.cut(
    data['liveness'],
    bins=[-0.01, 0.8, 1.0],
    labels=['pas live', 'live']
)

# Valence
df_cat_custom['valence_cat'] = pd.cut(
    data['valence'],
    bins=[-0.01, 0.5, 1.0],
    labels=['triste', 'joyeux']
)

# Tempo
df_cat_custom['tempo_cat'] = pd.cut(
    data['tempo'],
    bins=[-float('inf'), 100, 150, float('inf')],
    labels=['Lent', 'Modéré', 'Rapide']
)

# Duration (en secondes)
df_cat_custom['duration_cat'] = pd.cut(
    data['duration_s'],
    bins=[-0.01, 120, 240, float('inf')],
    labels=['court', 'moyen', 'long']
)
# Décennie de sortie de l'album
df_cat_custom['decade'] = (
    pd.to_datetime(data['track_album_release_date'], errors='coerce')
    .dt.year
    .dropna()
    .astype(int)
    .apply(lambda y: f"{(y // 10) * 10}s" if pd.notnull(y) else 'unknown')
)

# Fusionner les décennies 1950s et 1960s en une seule catégorie '1950s-1960s'
df_cat_custom['decade'] = df_cat_custom['decade'].replace({'1950s': '1950s-1960s', '1960s': '1950s-1960s'})

# Ajout des variables qualitatives existantes
df_cat_custom['mode'] = data['mode']
df_cat_custom['key'] = data['key']
df_cat_custom['playlist_genre'] = data['playlist_genre']

In [ ]:
# Subset for MCA with the new categorical variables
df_cat = df_cat_custom[
    [
        'popularity_cat', 'speechiness_cat', 'danceability_cat', 'energy_cat',
        'instrumentalness_cat', 'liveness_cat', 'valence_cat', 'tempo_cat',
        'duration_cat', 'decade', 'mode', 'playlist_genre'
    ]
].copy()

# Run MCA
mca = prince.MCA(n_components=2, n_iter=100, copy=True, check_input=True, engine='sklearn', random_state=1)
mca = mca.fit(df_cat)

# Get coordinates for the plot
coords = mca.column_coordinates(df_cat)

# Calculate cosine similarities for sizing
cosine_similarities = mca.column_cosine_similarities(df_cat)
sizes = cosine_similarities[0]**2 + cosine_similarities[1]**2

# Prepare data for plotting
plot_data = pd.DataFrame({
    'x': coords[0],
    'y': coords[1],
    'variable': [x.split('__')[0] for x in coords.index],
    'value': [x.split('__')[1] for x in coords.index],
    'size': sizes * 500  # Scale sizes for better visualization
})

# Create a color mapping for variables
color_list = sns.color_palette('tab20', n_colors=12)
variables = [
    'popularity_cat', 'speechiness_cat', 'danceability_cat', 'energy_cat',
    'instrumentalness_cat', 'liveness_cat', 'valence_cat', 'tempo_cat',
    'duration_cat', 'decade', 'mode', 'playlist_genre'
]
colors = dict(zip(variables, color_list))
plot_data['color'] = plot_data['variable'].map(colors)

# Create the plot
plt.figure(figsize=(12, 9))
scatter = sns.scatterplot(
    data=plot_data,
    x='x', 
    y='y',
    hue='variable',
    size='size',
    sizes=(50, 500),
    palette=colors,
    alpha=0.7
)

# Add annotations
for i, row in plot_data.iterrows():
    plt.text(
        row['x'] + 0.02, 
        row['y'] + 0.02, 
        row['value'], 
        fontsize=9,
        color='black'
    )

plt.title('MCA: Relations entre variables qualitatives audio', fontsize=16)
plt.xlabel(f'Dimension 1 ({mca.eigenvalues_summary.iloc[0, 1]})', fontsize=12)
plt.ylabel(f'Dimension 2 ({mca.eigenvalues_summary.iloc[1, 1]})', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.legend(title='Variable')
plt.tight_layout()
plt.show()


### Interprétation de l'Analyse des Correspondances Multiples (MCA)

#### Variables utilisées

Les variables qualitatives sélectionnées pour cette analyse sont :

* `playlist_genre` : Genre de la playlist (pop, rock, edm, rap, etc.)
* `mode` : Mode musical (majeur / mineur)
* `tempo_cat` : Catégorie de tempo (lent / rapide / modéré)
* `valence_cat` : Valence émotionnelle (joyeux / triste / neutre)
* `energy_cat`, `danceability_cat`, `instrumentalness_cat`, `speechiness_cat`, `liveness_cat`
* `duration_cat` : Longueur du morceau (court / moyen / long)
* `decade` : Décennie de sortie
* `popularity_cat` : Niveau de popularité (peu / moyen / très)
* `size` : Fréquence d'apparition (représentée par la taille des bulles)

---

### Résultats

#### 1. **Axes factoriels**

##### **Dimension 1 (8.15%)** :

Cet axe semble opposer :

* **À droite** : des genres et époques associés à une musique plus **live**, **peu dansante**, **rock** et **années 1970–1980**, souvent en **mode majeur**, avec une tendance à la **longueur**.
* **À gauche** : des genres comme **EDM**, **rap**, ou **pop des années 2010–2020**, associés à des morceaux plus **courts**, **instrumentaux**, **très populaires**, avec **paroles dominantes**, souvent **joyeux** et **mineur**.

##### **Dimension 2 (7.14%)** :

Cet axe reflète plutôt une opposition d’ambiance :

* **En haut** : Musiques plus **live**, **peu dansantes**, parfois **instrumentales**, associées à des décennies anciennes (1950s–1980s).
* **En bas** : Musiques plus **rapides**, **énergétiques**, **avec paroles dominantes**, parfois **r\&b** ou **pop**.

> La projection est inversée selon la dim 1 par rapport aux résultats sous R. Cela n'affecte pas l'interprétation, seule l’orientation spatiale est différente.

---

#### 2. **Proximité des modalités**

* Les genres **rap**, **latin**, **court**, **très populaire** et **paroles dominantes** sont regroupés → musique courte, populaire, axée sur le texte.
* Le **rock** est fortement associé au **mode majeur**, à la **décennie 1980**, à un **caractère live** et **peu dansant**.
* **EDM** est très proche de **instrumentale**, indiquant des morceaux sans paroles.
* Les catégories **"paroles dominantes"**, **"peu instrumental"**, et **"énergique"** sont toutes proches, suggérant que beaucoup de morceaux énergiques contiennent beaucoup de texte.
* **Pop des années 2010s et 2020s**, **dansants**, morceaux **moyennement long** se positionne plutôt au centre, ce qui reflète une certaine **polyvalence**.

---

#### 3. **Cos² (qualité de représentation)**

* Les modalités bien représentées par les deux axes (grosses bulles) incluent :

  * `rock`, `live`, `très populaire`, `joyeux`, `paroles dominantes`, `peu dansant`
* Les modalités avec des bulles plus petites comme `minor`, `pop`, `latin`, `r&b` sont moins bien représentées → elles nécessiteraient plus de dimensions pour être bien décrites.

---

### **Conclusion**

Cette MCA met en évidence des **clusters sémantiques clairs** entre les genres, l’époque, les caractéristiques émotionnelles et musicales :

* **Le rock des années 70–80** est clairement identifiable : live, peu dansant, majeur, long.
* **Le EDM et l’instrumental** vont de pair, souvent peu dansants, modernes, peu de texte.
* **Le rap** est court, populaire, avec des paroles dominantes, souvent dans un registre énergique mais pas exclusivement mineur ou majeur.
* **La pop des années 2010s–2020s** est très mixte : ni clairement joyeuse ni triste, ni très rapide ni lente → un profil "standard" qui touche un large public.


In [ ]:
# Choisir 4 artistes fréquents
top_artists = ['David Guetta', 'Ed Sheeran', 'Green Day', 'Billie Eilish', 'Eminem']

# Remplacer les autres artistes par 'Autre'
data['artist_subset'] = data['track_artist'].apply(lambda x: x if x in top_artists else 'Autre')

# Refaire le sous-ensemble avec artiste
df_cat1 = data[['playlist_genre', 'mode', 'key', 'artist_subset']].copy()

# Run MCA
mca = prince.MCA(n_components=2, n_iter=100, copy=True, check_input=True, engine='sklearn', random_state=1)
mca = mca.fit(df_cat1)

# Get coordinates for the plot
coords = mca.column_coordinates(df_cat1)

# Calculate cosine similarities for sizing
cosine_similarities = mca.column_cosine_similarities(df_cat1)
# Use the sum of cosine similarities as size parameter (for clearer visualization)
sizes = cosine_similarities[0]**2 + cosine_similarities[1]**2

# Prepare data for plotting
plot_data = pd.DataFrame({
    'x': coords[0],
    'y': coords[1],
    'variable': [x.split('__')[0] for x in coords.index],
    'value': [x.split('__')[1] for x in coords.index],
    'size': sizes * 500  # Scale sizes for better visualization
})

# Create a color mapping for variables
colors = {'playlist_genre': 'blue', 'mode': 'red', 'key': 'green', 'artist_subset': 'purple'}
plot_data['color'] = plot_data['variable'].map(colors)

# Create the plot
plt.figure(figsize=(10, 8))
scatter = sns.scatterplot(
    data=plot_data,
    x='x', 
    y='y',
    hue='variable',
    size='size',
    sizes=(50, 500),
    palette=colors,
    alpha=0.7
)

# Add annotations
for i, row in plot_data.iterrows():
    plt.text(
        row['x'] + 0.02, 
        row['y'] + 0.02, 
        row['value'], 
        fontsize=9,
        color='black'
    )

# Add styling
plt.title('MCA: Relationship between Genre, Mode and Key', fontsize=16)
plt.xlabel(f'Dimension 1 ({mca.eigenvalues_summary.iloc[0, 1]})', fontsize=12)
plt.ylabel(f'Dimension 2 ({mca.eigenvalues_summary.iloc[1, 1]})', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.legend(title='Variable Type')

# Show plot
plt.tight_layout()
plt.show()


### Distances artistes-genres (MCA)
Nous avons décidé d'affiner notre MCA en y intégrant certains artistes pour mieux comprendre leurs relations avec les genres musicaux. Nous avons sélectionné 6 artistes : **David Guetta**, **Ed Sheeran**, **Green Day**, **Billie Eilish**, **Eminem** et un groupe d'artistes diversifié nommé **Autre**. Afin d'analyser la proximité entre ces artistes et les genres musicaux, nous avons calculé les distances euclidiennes au carré entre chaque artiste et les genres issus de l'Analyse des Correspondances Multiples (MCA).

#### Interprétations individuelles

- **David Guetta** est très proche du genre **EDM**, ce qui est cohérent avec son positionnement d’artiste électro. Il est aussi relativement proche du genre `pop` et `rock`, ce qui peut s’expliquer par ses nombreuses collaborations avec des chanteurs populaires.

- **Ed Sheeran** est modérément proche des genres **pop** et **edm**, ce qui reflète sa diversité musicale et ses morceaux mêlant acoustique et sons plus produits.

- **Green Day** est relativement proche du genre **rock**, ce qui confirme son ancrage dans ce style. Il est plus éloigné des autres genres, ce qui le distingue nettement stylistiquement dans cette analyse.

- **Billie Eilish** est située à mi-distance de plusieurs genres (`pop`, `edm`, `rock`), ce qui traduit une position hybride. Elle n’est pas fortement associée à un genre unique, ce qui est cohérent avec son style singulier et difficile à catégoriser.

- **Eminem** montre une **affinité forte avec le genre Rap**, avec une distance minimale comparée aux autres genres. Il est largement séparé des genres comme rock ou pop, ce qui reflète bien son style musical distinct.

- **Autre** est, comme attendu, proche du centre, ce qui traduit une position moyenne, résultant de l’agrégation d’artistes divers aux profils variés.

### Conclusion

L’analyse des distances artistes-genres via la MCA confirme plusieurs intuitions sur les positionnements stylistiques des artistes. Elle permet également de détecter des cas hybrides (comme Billie Eilish)
Elle enrichit l’interprétation visuelle de l’ACM en objectivant la notion de "proximité" par une mesure quantitative.

## NMF

La Factorisation Matricielle Non-Négative (NMF) est une technique de réduction de dimension qui permet de décomposer une matrice en deux matrices de facteurs non négatifs. Dans le contexte de l'analyse des données musicales, NMF est particulièrement utile pour identifier des motifs latents ou des profils musicaux à partir des caractéristiques audio.

### Objectif de la NMF
L'objectif de la NMF dans ce projet est de découvrir des profils musicaux latents qui peuvent représenter des styles ou des genres musicaux spécifiques. En factorisant les données audio, nous espérons extraire des caractéristiques communes qui peuvent être utilisées pour la recommandation de musique ou pour mieux comprendre la structure des genres musicaux.

In [ ]:
audio_features = [
    'danceability', 'energy', 'loudness', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness',
    'valence', 'tempo', 'duration_s'
]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

# 1. Sélection des features audio
X = data_songs[audio_features].copy()

# 2. Normalisation min-max (important pour la NMF)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 3. Tester plusieurs valeurs de r (nombre de composants)
errors = []
r_values = range(2,11, 2)

for r in r_values:
    model = NMF(n_components=int(r), init='nndsvda', random_state=42, max_iter=500)
    W = model.fit_transform(X_scaled)
    H = model.components_
    reconstruction = np.dot(W, H)
    error = np.linalg.norm(X_scaled - reconstruction, 'fro')  # norme de Frobenius
    errors.append(error)

# 4. Tracer la courbe de l’erreur de reconstruction
plt.figure(figsize=(10, 6))
plt.plot(r_values, errors, marker='o')
plt.title("Erreur de reconstruction vs nombre de composants (r)")
plt.xlabel("Nombre de composants (r)")
plt.ylabel("Erreur de reconstruction (norme de Frobenius)")
plt.grid(True)
plt.show()


On remarque que l'erreur de reconstruction décroit jusqu'à notre nombre variables total, on choisit arbitrairement de prendre 6 facteurs pour la suite de l'analyse, ce qui va nous donner 6 profils musicaux.

In [ ]:
# 3. Application de la NMF
nmf_model = NMF(n_components=6, init='nndsvda', random_state=42, max_iter=500)
W = nmf_model.fit_transform(X_scaled)
H = nmf_model.components_

# 4. Créer un DataFrame pour la matrice H
H_df = pd.DataFrame(H, columns=audio_features)
H_df.index = [f'Profil {i+1}' for i in range(H.shape[0])]

# 5. Affichage de la matrice H
print("Matrice H (profils latents définis par les variables audio) :")
display(H_df)

# 6. Visualisation : contribution des variables à chaque profil
plt.figure(figsize=(12, 6))
sns.heatmap(H_df, annot=True, cmap='YlGnBu', fmt=".2f")
plt.title("Profils latents musicaux détectés par NMF (matrice H)")
plt.xlabel("Caractéristiques audio")
plt.ylabel("Profils NMF")
plt.tight_layout()
plt.show()

### Dataset utilisé
Nous avons décidé d'utiliser le dataframe `data_songs` ne contenant que les morceaux une seule fois dans le dataset, pour éviter de biaiser les résultats. 

### Variables utilisées

Les variables sélectionnées pour cette NMF sont les caractéristiques audio continues suivantes :

* `danceability`, `energy`, `loudness`, `speechiness`, `acousticness`, `instrumentalness`, `liveness`, `valence`, `tempo`, `duration_s`.

Nous avons exclu la variable `track_popularity` car la NMF est très sensible aux valeurs extrêmes et cette variable a une distribution très héterogène. De plus, nous avons exclu les variables catégorielles et les variables de texte, car la NMF est conçue pour fonctionner sur des données numériques continues.

### Résultats

#### 1. Profils latents détectés

L’analyse a révélé **6 profils musicaux latents** à partir des données. Chaque profil est défini par une combinaison unique de caractéristiques audio. Voici leur interprétation :

* **Profil 1 : Profil Énergique-Intense**

  * Caractérisé par de **très fortes valeurs en `energy`, `loudness` et `tempo` et `duration`**.
  * Faible en composantes émotionnelles (`valence`), vocales et instrumentales.
  * Ce profil pourrait correspondre à des morceaux **très dynamiques et bruyants**, typiques de l’**EDM** (house, electro, techno) ou du **hard-rock**/**metal**.

* **Profil 2 : Profil Acoustique-Chant**

  * Dominé par une forte **`acousticness`**.
  * Ce profil, presente **`instrumentalness`=0**, et un faible **`speechiness`** mais différent de zero.
  * Ce profil regroupe possiblement des morceaux **acoustiques et chantés**.

* **Profil 3 : Profil Instrumental**

  * Très forte **`instrumentalness`** avec très peu d'autres caractéristiques.
  * Correspond clairement à des morceaux **purement instrumentaux**, probablement **ambient**, **classique** ou **bande-son**.

* **Profil 4 : Profil Émotionnel-Valence**

  * Faible en toutes dimensions sauf une très forte **`valence`**.
  * Ce profil regroupe des morceaux **émotionnellement très positifs**, avec une ambiance **joyeuse ou euphorique**, indépendamment du tempo ou de l’énergie.

* **Profil 5 : Profil Vocal-Live**

  * Faibles valeurs générales sauf une **forte `liveness`** et une **`speechiness`** marquée.
  * Peut représenter des morceaux **live**, **rap**, ou des chansons **parlées/spoken word**.

* **Profil 6 : Profil Rap-Rythmé**

  * Forte `danceability`, `speechiness` et modérée `energy`.
  * Ce profil semble capturer des morceaux **rythmés et très vocaux**, typiques du **rap**, **hip-hop**, voire certains morceaux **R\&B urbains**. Ils pourraient aussi correspondre à des morceaux **pop** et **latino** dans une certaine mesure.

---

Pour chaque profil, nous allons desormais regarder les morceaux qui y contribuent le plus fortement, pour confirmer ces resultats.

In [ ]:
# Get the top 2 tracks for each NMF profile
n_profiles = 6  # Number of profiles from our NMF model
top_n = 2      # Number of top tracks to display per profile

# Create a dataframe with track information and profile weights
profile_weights = pd.DataFrame(W, index=data_songs.index)
profile_weights.columns = [f'Profile_{i+1}' for i in range(n_profiles)]

# For each profile, find the tracks with the highest weights
results = []

for profile_idx in range(n_profiles):
    profile_name = f'Profile_{profile_idx+1}'
    
    # Get top tracks for this profile
    top_indices = profile_weights[profile_name].nlargest(top_n).index
    
    # Extract relevant information for these tracks
    for idx in top_indices:
        track_info = {
            'Profile': profile_name,
            'Track Name': data.loc[idx, 'track_name'],
            'Artist': data.loc[idx, 'track_artist'],
            'Genre': data.loc[idx, 'playlist_genre'],
            'Subgenre': data.loc[idx, 'playlist_subgenre'],
            'Weight': profile_weights.loc[idx, profile_name]
        }
        
        # Add audio features for context
        for feature in audio_features:
            track_info[feature] = data.loc[idx, feature]
            
        results.append(track_info)

# Create a dataframe with the results and display it
top_tracks_df = pd.DataFrame(results)

# Sort by profile and weight
top_tracks_df = top_tracks_df.sort_values(['Profile', 'Weight'], ascending=[True, False])
display(top_tracks_df)

**Interprétation :**

On retrouve des resultats cohérents avec les profils identifiés précédemment. Seul `Sandstorm` de Darude qui est censé être un morceau EDM est classé dans le profil 2, ce qui est surprenant. Mais en regardant la caractéristique `instrumentalness`, on remarque qu'il est très proche de 1, ce qui signifie que le morceau est classé comme très intrumental. Il est donc normal qu'il soit classé dans le profil 2.

---

On peut desormais regarder pour certains artistes quels sont les profils musicaux qui leur correspondent le mieux. On va regarder les tracks des artistes suivants : **David Guetta**, **Ed Sheeran**, **Green Day**, **Billie Eilish** et **Eminem**.

In [ ]:
# Choisir 4 artistes fréquents
top_artists = ['David Guetta', 'Ed Sheeran', 'Green Day', 'Billie Eilish', 'Eminem']

# Create a dataframe with profile weights for each track
profile_weights = pd.DataFrame(W, index=data_songs.index)
profile_weights.columns = [f'Profile_{i+1}' for i in range(n_profiles)]

# Merge with original data to get artist info
artist_profiles = pd.concat([data_songs[['track_artist', 'track_name']], profile_weights], axis=1)

# Initialize a dictionary to store results
artist_distribution = {}

# For each artist, analyze their tracks
for artist in top_artists:
    # Filter tracks by this artist
    artist_tracks = artist_profiles[artist_profiles['track_artist'] == artist]
    
    if len(artist_tracks) == 0:
        print(f"No tracks found for {artist}")
        continue
    
    # Calculate average profile weight for this artist
    avg_profile_weights = artist_tracks[[f'Profile_{i+1}' for i in range(n_profiles)]].mean()
    
    # Find dominant profile for each track
    dominant_profiles = artist_tracks[[f'Profile_{i+1}' for i in range(n_profiles)]].idxmax(axis=1).value_counts()
    
    # Store results
    artist_distribution[artist] = {
        'track_count': len(artist_tracks),
        'avg_weights': avg_profile_weights,
        'dominant_profiles': dominant_profiles
    }

# Visualize the results
fig, axes = plt.subplots(len(top_artists), 2, figsize=(18, 4*len(top_artists)))

for i, artist in enumerate(top_artists):
    if artist not in artist_distribution:
        continue
    
    # Average profile weights
    artist_distribution[artist]['avg_weights'].plot(
        kind='bar', 
        ax=axes[i, 0], 
        color='skyblue',
        title=f"{artist}: Average Profile Weights (n={artist_distribution[artist]['track_count']} tracks)"
    )
    axes[i, 0].set_ylabel("Weight")
    axes[i, 0].set_xlabel("NMF Profile")
    
    # Distribution of dominant profiles
    if len(artist_distribution[artist]['dominant_profiles']) > 0:
        artist_distribution[artist]['dominant_profiles'].plot(
            kind='bar', 
            ax=axes[i, 1], 
            color='coral',
            title=f"{artist}: Dominant Profile Distribution"
        )
        axes[i, 1].set_ylabel("Number of Tracks")
        axes[i, 1].set_xlabel("Dominant NMF Profile")

plt.tight_layout()
plt.show()

# Print a summary
print("\nArtist Profile Summary:")
print("=" * 50)
for artist in top_artists:
    if artist not in artist_distribution:
        continue
    
    print(f"\n{artist} ({artist_distribution[artist]['track_count']} tracks)")
    print("-" * 30)
    
    # Get the dominant profile(s)
    if len(artist_distribution[artist]['dominant_profiles']) > 0:
        top_profile = artist_distribution[artist]['dominant_profiles'].index[0]
        top_count = artist_distribution[artist]['dominant_profiles'].iloc[0]
        top_percent = (top_count / artist_distribution[artist]['track_count']) * 100
        
        print(f"Primary Profile: {top_profile} ({top_percent:.1f}% of tracks)")
    
    # Get the highest weight profile on average
    top_avg_profile = artist_distribution[artist]['avg_weights'].idxmax()
    top_avg_weight = artist_distribution[artist]['avg_weights'].max()
    
    print(f"Highest Average Weight: {top_avg_profile} (weight: {top_avg_weight:.3f})")

In [ ]:
# Get all tracks by Green Day
green_day_tracks = data_songs[data_songs['track_artist'] == 'Green Day']

# Create a dataframe with profile weights for each Green Day track
profile_weights = pd.DataFrame(W, index=data_songs.index)
profile_weights.columns = [f'Profile_{i+1}' for i in range(n_profiles)]

# Merge track information with profile weights
green_day_profiles = pd.concat([green_day_tracks[['track_name']], 
                               profile_weights.loc[green_day_tracks.index]], axis=1)

# Add the dominant profile column
green_day_profiles['Dominant_Profile'] = green_day_profiles[[f'Profile_{i+1}' for i in range(n_profiles)]].idxmax(axis=1)

# Sort by dominant profile and then by the weight of that profile (descending)
green_day_profiles = green_day_profiles.sort_values(['Dominant_Profile', 'track_name'])

# Display the tracks with their profile information
display(green_day_profiles[['track_name', 'Dominant_Profile'] + 
                          [f'Profile_{i+1}' for i in range(n_profiles)]])


### Artistes et profils musicaux

En dehors de **Green Day**, ayant des morceaux partagés entre les profiles 4, 5 et 6, **les autres artistes appartiennent tous au profil 6**. Pour ce qui est de **Green Day**, les morceaux qui lui sont attribués dans le profil 4 sont en effet des morceaux joyeux et gais comme *American Idiot* ou *Basket Case* du moins dans la direction artistique.

Les morceaux qui lui sont attribués dans le **profil 5** sont des morceaux **qui a priori ne sont pas des versions live** mais l'algorithme de Spotify en charge de detecter les versions live **a du se tromper**. Si cette variable présente des valeurs non représentatives, il serait intéressante de répéter l'analyse sans cette variable.

Pour ce qui est des morceaux qui lui sont attribués dans le **profil 6**, ce sont des morceaux pop-rock **plutot équilibrés** comme *Boulevard of Broken Dreams* ou *Wake Me Up When September Ends*.

---

On peut conclure sur le fait que ce profil 6 regroupe plus de styles musicaux que les autres profils. Il semble être **le profil le plus généraliste**, et regroupe des morceaux de plusieurs genres musicaux.

Il serait utile de faire un clustering sur les profils musicaux pour voir si on peut regrouper les morceaux en fonction de leurs caractéristiques audio, particulièrement sur le profil 6.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import colors
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer

In [ ]:
kmeans = KMeans(init='k-means++', n_init='auto', random_state=42)
visualizer = KElbowVisualizer(kmeans, k=(2, 12))
visualizer.fit(W)
visualizer.show()

In [ ]:
# Perform K-Means clustering on W with 3 clusters
kmeans_w = KMeans(n_clusters=4, random_state=42, n_init='auto')
cluster_labels = kmeans_w.fit_predict(W)

# Add cluster labels to a DataFrame for easier handling
clustered_data = data_songs.copy()
clustered_data['kmeans_cluster'] = cluster_labels

#Confusion Matrix: K-Means clusters vs. NMF profiles

# Determine the dominant NMF profile for each track
# profile_weights was calculated in CELL INDEX 50 and has data_songs.index
dominant_nmf_profile = profile_weights.idxmax(axis=1)
clustered_data['dominant_nmf_profile'] = dominant_nmf_profile

# Create confusion matrix
# Ensure the labels match the unique values in dominant_nmf_profile and kmeans_cluster
nmf_labels = clustered_data['dominant_nmf_profile'].unique()
kmeans_labels = sorted(clustered_data['kmeans_cluster'].unique())

# Create a contingency matrix instead of a confusion matrix
contingency_matrix = pd.crosstab(clustered_data['dominant_nmf_profile'], clustered_data['kmeans_cluster'])

# Convert the contingency matrix to a DataFrame for visualization
cm_nmf_df = contingency_matrix

plt.figure(figsize=(10, 7))
sns.heatmap(cm_nmf_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix: K-Means Clusters vs. Dominant NMF Profiles')
plt.ylabel('Dominant NMF Profile')
plt.xlabel('K-Means Cluster')
plt.show()

#Confusion Matrix: K-Means clusters vs. Playlist Genres

# Get playlist genres for tracks in data_songs from the original 'data' DataFrame
playlist_genres_for_W = data.loc[data_songs.index, 'playlist_genre'] # Usage of data.loc to ensure correct indexing
clustered_data['playlist_genre'] = playlist_genres_for_W



**Interprétation :**

L’analyse des clusters obtenus à partir de la **matrice W de la NMF** révèle des profils très contrastés :

* Les **profils 1 à 4 sont clairement définis et homogènes** : ils se retrouvent **presque exclusivement dans un seul cluster (le cluster 3)**, ce qui indique une **cohérence interne forte** et une bonne séparation de ces profils par la NMF.

* Le **profil 5** montre une **certaine diversité**, mais reste **partiellement concentré dans le cluster 1**, suggérant une structure intermédiaire : ni totalement homogène, ni complètement dispersé.

* Le **profil 6** est **largement hétérogène**, avec des morceaux **répartis sur l’ensemble des 5 clusters**, ce qui confirme plusieurs points importants :

  * Ce profil capture **un groupe de morceaux très variés**, sans dominante claire.
  * Il reflète probablement **plusieurs sous-genres ou styles musicaux imbriqués**.
  * Sa **taille très supérieure** à celle des autres profils suggère que la **NMF n’a pas réussi à découper efficacement cet ensemble**, ce qui peut être dû à une **grande diversité sonore** ou à une **structure non linéaire difficile à factoriser**.

---

**Bilan :**

L’analyse met en évidence que **seul le profil 6 mérite une exploration plus approfondie**. On peut maintenant **zoomer sur ce profil**, et **interpréter les clusters qui le composent** en examinant :

* La répartition des **genres** et **sous-genres musicaux**.
* La structure des **caractéristiques audio** propres à chaque cluster.

Cela permettra d’identifier si des **sous-groupes cohérents** émergent au sein de ce profil complexe.


In [ ]:
# Identifier le profil dominant pour chaque morceau
dominant_profile = W.argmax(axis=1)  # Renvoie l'indice du profil dominant (0 à 5)

# Filtrer uniquement ceux dont le profil dominant est le profil 6 (index 5)
profile_6_indices = np.where(dominant_profile == 5)[0]
W_profile_6 = W[profile_6_indices]

kmeans = KMeans(init='k-means++', n_init='auto', random_state=42)
visualizer = KElbowVisualizer(kmeans, k=(2, 12))
visualizer.fit(W_profile_6)
visualizer.show()

In [ ]:
# Filtrer uniquement les données correspondant au profil 6
profile_6_indices = np.where(dominant_profile == 5)[0]  # Indices des morceaux avec profil dominant 6
W_profile_6 = W[profile_6_indices]  # Matrice W pour le profil 6
data_songs_profile_6 = data_songs.iloc[profile_6_indices]  # Filtrer les morceaux correspondants

# Effectuer le clustering K-Means sur W profil 6
kmeans_profile_6 = KMeans(n_clusters=6, random_state=42, n_init='auto')
cluster_labels_profile_6 = kmeans_profile_6.fit_predict(W_profile_6)

# Ajouter les étiquettes de cluster au DataFrame filtré
data_songs_profile_6['kmeans_cluster'] = cluster_labels_profile_6

# Créer une matrice de contingence pour les genres de playlist
playlist_genres_for_W_profile_6 = data.loc[data_songs_profile_6.index, 'playlist_genre']
data_songs_profile_6['playlist_genre'] = playlist_genres_for_W_profile_6

# Matrice de contingence : clusters K-Means vs genres de playlist
cm_genre_df_profile_6 = pd.crosstab(data_songs_profile_6['playlist_genre'], data_songs_profile_6['kmeans_cluster'])

# Visualisation de la matrice de contingence
plt.figure(figsize=(12, 8))
sns.heatmap(cm_genre_df_profile_6, annot=True, fmt='d', cmap='Greens')
plt.title('Confusion Matrix: K-Means Clusters vs. Playlist Genres (Profile 6)')
plt.ylabel('Actual Playlist Genre')
plt.xlabel('K-Means Cluster')
plt.show()


In [ ]:
# Créer une matrice de contingence pour les sous-genres de playlist
playlist_subgenres_for_W_profile_6 = data.loc[data_songs_profile_6.index, 'playlist_subgenre']
data_songs_profile_6['playlist_subgenre'] = playlist_subgenres_for_W_profile_6

# Matrice de contingence : clusters K-Means vs sous-genres de playlist
cm_subgenre_df_profile_6 = pd.crosstab(data_songs_profile_6['playlist_subgenre'], data_songs_profile_6['kmeans_cluster'])

# Visualisation de la matrice de contingence
plt.figure(figsize=(12, 8))
sns.heatmap(cm_subgenre_df_profile_6, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix: K-Means Clusters vs. Playlist Subgenres (Profile 6)')
plt.ylabel('Actual Playlist Subgenre')
plt.xlabel('K-Means Cluster')
plt.show()

#### **Interprétation :**

L’analyse des **clusters extraits du profil 6** (via NMF + k-means) révèle des résultats **globalement hétérogènes**, notamment lorsqu’on les compare aux **genres de playlists** : les morceaux sont **fortement dispersés entre les clusters**, ce qui rend l’interprétation difficile.

Cependant, en se concentrant sur les **sous-genres musicaux**, **certains motifs apparaissent** et permettent de mieux comprendre la structure interne des clusters. Voici les principales observations :

---

##### **Une dispersion marquée des morceaux electro-House**

Les morceaux d'**electro-house** (et dérivés) sont **présents dans presque tous les clusters**, **à l’exception du cluster 5**. Cela indique que :

* Ces morceaux présentent **un large spectre de sonorités**, les rendant difficiles à regrouper de manière cohérente.
* Cela suggère que la **NMF + k-means n’a pas réussi à isoler une structure claire** pour ce style, à la différence d’autres genres plus homogènes comme le hard-rock.

---

##### **Clusters 0 et 3 : Des profils rap différenciés**

Ces deux clusters sont dominés par des **sous-genres du rap**, mais selon des nuances distinctes :

* Le **cluster 0** contient principalement du **gangster rap** et du **southern hip-hop**, qui y sont **deux fois plus représentés** que les autres styles de rap.
* Le **cluster 3**, quant à lui, est **plus équilibré** : on y trouve des parts similaires de **trap**, **hip-hop**, **southern hip-hop** et **gangster rap**, avec aussi une présence notable de **latin hip-hop**.

Ces différences montrent que **le modèle distingue finement certaines nuances à l’intérieur même du spectre rap**.

---

##### **Cluster 1 : Une dominante pop**

À première vue, le **cluster 1** semble très hétérogène. Toutefois, en examinant les sous-genres les plus fréquents, une tendance claire émerge :

* On y retrouve majoritairement des styles tels que **pop**, **pop-rock**, **post-teen pop**, **dance pop**, **latin pop**, **pop-edm**, et **indie poptimism**.

Cela s’explique par la **polyvalence intrinsèque des morceaux pop**, souvent présents dans des playlists de styles variés — une **caractéristique reflétée dans leur regroupement** ici.

---

##### **Cluster 2 : Le noyau dur du rock**

Ce cluster est le **plus homogène** de tous :

* Il regroupe en majorité des morceaux de **hard-rock**, ainsi que des variantes comme le **pop-rock**, **rock alternatif**, **indie poptimism**, ou encore **permanent wave**.

Ce regroupement suggère que **les morceaux de rock (notamment hard) présentent des caractéristiques audio distinctes**, bien captées par la factorisation NMF et le groupement en clusters.

---

##### **Cluster 5 : Des morceaux hybrides et alternatifs**

Ce dernier cluster contient principalement des morceaux de type :

* **indie poptimism**
* **neo soul**
* **urban contemporary**

Il semble regrouper des morceaux **plus hybrides, subtils ou alternatifs**, peut-être à la **frontière entre plusieurs styles** — ce qui pourrait expliquer leur isolement dans ce cluster spécifique.

---

##### **Bilan** :

Malgré une **forte hétérogénéité globale du profil 6 de la NMF**, l’analyse fine des **sous-genres** permet d’**extraire des structures locales** intéressantes. En particulier :

* Les **morceaux de hard-rock** sont **clairement isolés**.
* Les **raps** sont répartis mais **organisés selon des affinités fines**.
* Les **morceaux pop**, bien que omniprésents, montrent une **cohérence interne dans leur dispersion**.
* Enfin, la **présence massive de l’electro-house dans presque tous les clusters** met en lumière **les limites du k-means à capturer les spécificités de ce style à travers la projection de la NMF**, ou bien la **diversité sonore intrinsèque** de ce genre.



# Clustering 

Dans cette partie, nous allons mettre en œuvre différentes méthodes de clustering afin d’identifier des groupes homogènes de morceaux au sein de notre jeu de données. L’objectif est de regrouper les morceaux présentant des caractéristiques similaires, sans utiliser d’information sur leur genre ou sous-genre, afin de révéler des structures ou des tendances cachées dans les données.  
Nous appliquerons plusieurs algorithmes de clustering non supervisé, tels que **K-Means**, le **mélange gaussien (GMM)** et la **classification ascendante hiérarchique (CAH)**. Nous comparerons ensuite les résultats obtenus et analyserons les profils des clusters identifiés, en les reliant aux variables musicales et aux genres présents dans le jeu de données.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from matplotlib import colors
from yellowbrick.cluster import SilhouetteVisualizer, KElbowVisualizer
import seaborn as sns

>**IMPORTANT**  
> Dans ce notebook nous avons fait le choix de faire le clustering pur sur le jeu *data_songs* et ensuite nous allons projeter les clusters obtenus sur les genres des playlists. Ce choix a été fait afin de se concentrer principalement sur les caractéristiques audios des morceaux afin d'obtenir des profils musicaux. 

## Projection factorielle des inividus (ACP)
On va d'abord afficher la représentation factorielle des individus pour pouvoir voir si on est capable de distinguer des classes naturelles. Pour cela, nous effectuons comme dans la partie précédente une ACP sur le jeu *data_songs*.

In [ ]:
# Sélection des variables quantitatives de data_songs
X = data_songs.select_dtypes(include=['int64', 'float64'])

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA : on garde 10 composantes pour l'affichage de la variance expliquée
pca_songs = PCA(n_components=10)
pca_songs_coords = pca_songs.fit_transform(X_scaled)

explained_variance = pca_songs.explained_variance_ratio_

eig = pd.DataFrame({
    "Dimension": [f"CP{i+1}" for i in range(len(explained_variance))],
    "Variance": np.round(pca_songs.explained_variance_, 2),
    "% variance expliquée": np.round(explained_variance * 100, 1),
    "% variance cumulée": np.round(np.cumsum(explained_variance) * 100, 1)
})

fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(1, 2, 1)
ax1.bar(range(1, 11), explained_variance[:10] * 100, align='center', color='coral', ecolor='black')
ax1.set_xticks(range(1, 11))
ax1.set_xlabel("Composante principale")
ax1.set_ylabel("% Variance expliquée")
ax1.set_title("Pourcentage de variance expliqué", fontsize=16)

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(range(1, 11), np.cumsum(explained_variance[:10]) * 100, color='coral', marker='o')
ax2.axhline(80, color='grey', linestyle='--', alpha=0.5, label='80%')
ax2.set_xlabel("Nombre de composantes")
ax2.set_ylabel("% Variance expliquée cumulée")
ax2.set_title("Pourcentage de variance expliqué cumulé", fontsize=16)
ax2.legend()

fig.suptitle("Résultat ACP sur data_songs", fontsize=20)
plt.tight_layout()
plt.show()

On obtient les mêmes résultats que pour l'analyse précédente. Nous allons donc faire le choix de faire nos analyses que sur les **3 premières composantes principales**.

In [ ]:
pca_songs_df = pd.DataFrame(pca_songs_coords[:, :3], columns=['CP1', 'CP2', 'CP3'])
combinations = [('CP1', 'CP2'), ('CP2', 'CP3'), ('CP1', 'CP3')]

# On récupère le genre des morceaux depuis le jeu de données initial
pca_songs_with_genre = pca_songs_df.copy()
pca_songs_with_genre['playlist_genre'] = data.loc[data_songs.index, 'playlist_genre'].values

fig, axes = plt.subplots(2, 3, figsize=(24, 12))

# Ligne 1 : scatterplots sans couleur par genre
for i, (x, y) in enumerate(combinations):
    sns.scatterplot(
        data=pca_songs_df,
        x=x,
        y=y,
        color='steelblue',
        s=10,
        alpha=0.5,
        ax=axes[0, i]
    )
    axes[0, i].set_title(f'Projection PCA : {x} vs {y}')
    axes[0, i].axhline(0, linestyle='--', color='gray', linewidth=0.5)
    axes[0, i].axvline(0, linestyle='--', color='gray', linewidth=0.5)

# Ligne 2 : scatterplots colorés par genre
for i, (x, y) in enumerate(combinations):
    show_legend = (i == 0) 
    sns.scatterplot(
        data=pca_songs_with_genre,
        x=x,
        y=y,
        hue='playlist_genre',
        palette='tab10',
        s=10,
        alpha=0.6,
        ax=axes[1, i],
        legend=show_legend
    )
    axes[1, i].set_title(f'Projection PCA : {x} vs {y} (par genre)')
    axes[1, i].axhline(0, linestyle='--', color='gray', linewidth=0.5)
    axes[1, i].axvline(0, linestyle='--', color='gray', linewidth=0.5)

plt.tight_layout()
plt.show()

Nous projetons d’abord les morceaux sur les trois premiers axes principaux sans distinction de genre, afin d’observer si des regroupements naturels apparaissent dans l’espace réduit.  
Ensuite, nous colorons les points selon le genre musical associé à chaque morceau pour voir si cette information permet de distinguer des groupes plus nets.

Cependant, que ce soit sans ou avec la distinction de genre, aucune séparation claire ou cluster naturel n’apparaît visuellement dans les différents plans factoriels. Cela suggère que les genres sont fortement mélangés dans l’espace des composantes principales. On réussit néanmoins à distinguer que le style **EDM** à l'air de posséder des valeurs plus extrêmes que les autres genres (moins mélangés).  
Cette observation motive l’utilisation de méthodes de clustering non supervisées pour aller plus loin dans l’identification de groupes homogènes.

## K-Means
Avant d’appliquer l’algorithme de clustering K-Means, il est nécessaire de choisir le nombre optimal de clusters à utiliser.
Comme l’exploration visuelle précédente ne permet pas d’identifier clairement un nombre naturel de groupes, nous avons recours à la méthode du coude.  
Cette méthode consiste à faire varier le nombre de clusters et à observer l’évolution de l’inertie (distortion) pour déterminer le point à partir duquel ajouter des clusters n’apporte plus de gain significatif.
Le code suivant permet de visualiser ce critère et d’identifier le nombre de clusters optimal pour K-Means :

In [ ]:
kmeans = KMeans(init='k-means++', n_init='auto', random_state=42)
visualizer = KElbowVisualizer(kmeans, k=(2, 12), metric='distortion')
visualizer.fit(pca_songs_df.values)
visualizer.show()

D'après le critère, le nombre de classes optimal pour cette méthode de clustering est **5 classes**. Nous allons donc conserver ce nombre pour la suite de notre analyse. 

In [ ]:
K = 5 # nombre de clusters optimal
kmeans = KMeans(n_clusters=K, init='k-means++', n_init='auto', max_iter=300, random_state=42)
kmeans.fit(pca_songs_df)
pca_songs_df['cluster'] = kmeans.labels_

In [ ]:
clusters = sorted(pca_songs_df['cluster'].unique())
colors = plt.colormaps['Set2'].resampled(len(clusters))

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].set_title('CP1 vs CP2')
axes[1].set_title('CP2 vs CP3')
axes[2].set_title('CP1 vs CP3')

for i, cluster in enumerate(clusters):
    subset = pca_songs_df[pca_songs_df['cluster'] == cluster]
    axes[0].scatter(subset['CP1'], subset['CP2'], label=f'Cluster {cluster}', alpha=0.6, s=10, color=colors(i))
    axes[1].scatter(subset['CP2'], subset['CP3'], label=f'Cluster {cluster}', alpha=0.6, s=10, color=colors(i))
    axes[2].scatter(subset['CP1'], subset['CP3'], label=f'Cluster {cluster}', alpha=0.6, s=10, color=colors(i))

for ax in axes:
    ax.legend(title='Cluster')
    ax.grid(True)

plt.tight_layout()
plt.show()

Dans chacune des représentations, nos clusters semblent être assez distincts. Il ne semble pas y avoir beaucoup de mélange de classes. 

Certaines classes semblent avoir une taille différente, pouvant indiquer un éventuel déséquilibre.

Afin d'avoir une analyse plus complète de ces clusters, nous allons étudier les variables explicatives du jeu de données avec les clusters obtenus. 

**Étude des autres variables quantitatives avec nos clusters**  
On a plusieurs manières d'étudier les variables quantitatives dans nos clusters, soit avec des boxplots (assez visuels), soit avec une `heatmap`, cela permet d'avoir les moyennes centralisées et on a un visuel global de répartitions dans les clusters.

In [ ]:
data_songs['cluster'] = pca_songs_df['cluster'].values

In [ ]:
features = [
    'danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
    'instrumentalness', 'liveness', 'valence', 'tempo', 'track_popularity', 
    'release_year', 'duration_s'
]

n_cols = 4
n_rows = (len(features) + n_cols - 1) // n_cols  

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.boxplot(x='cluster', y=feature, data=data_songs, ax=axes[i], showfliers=False)
    axes[i].set_title(f'{feature} par cluster KMeans')

for j in range(len(features), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

Voici l'analyse que nous pouvons fournir à l'aide de la lecture des boxplots.

**Cluster 0 :**  
* Morceaux avec une *danceability* moyenne, mais avec une *energy*, *loudness* et *tempo* très élevé.
* L'*acousticness* et la *valence* sont assez faibles.
* <u>**Profil**</u> **:** Morceaux dynamiques et puissants qui sont peu acoustiques et joyeux.

**Cluster 1 :**
* Haute *danceability* mais le reste des variables ont un profil plutôt moyen. 
* <u>**Profil**</u> **:** Morceaux très dansants, mais modérément dynamiques. 

**Cluster 2 :**
* L'*acousticness* et l'*instrumentalness* sont élevés, pour le reste, tout est assez faible. 
* <u>**Profil**</u> **:** Morceaux calmes, acoustiques et peu joyeux. 

**Cluster 3 :**
* La *danceability* est très élevée, de même que pour *energy*, *loudness* et *valence*. 
* <u>**Profil**</u> **:** Morceaux dynamiques, puissants, dansants et joyeux. 

**Cluster 4 :**
* La *danceability* est élevé, ainsi qu'*energy*, *loudness* et *instrumentalness*. La *valence* est plutôt moyenne. 
* La durée des morceaux de ce cluster est plus longue que pour les autres. Et la popularité est très faible. 
* <u>**Profil**</u> **:** Morceaux instrumentaux, dynamiques, qui sont longs et peu populaires. 

D'après ces premières analyses, on peut penser aux genres musicaux suivants pour chaque cluster : 
* Cluster 0 : rock, électro. 
* Cluster 1 : pop, hip-hop.
* Cluster 2 : musiques d'ambiances, ballades.
* Cluster 3 : musiques très festives, comme la pop festive, dance.
* Cluster 4 : musiques très instrumentales comme les bandes originales de films ou de l'électro. 

Nous allons ensuite afficher la proportion des genres dans les clusters. Cela va nous permettre d'éventuellement confirmer nos analyses faites précédemment. 

Pour valider et affiner l’interprétation de nos clusters, nous avons croisé l’appartenance aux clusters avec les genres et sous-genres musicaux à l’aide de tableaux croisés (crosstab) et de heatmaps. Cela permet de visualiser la répartition des genres au sein de chaque cluster et d’identifier d’éventuelles correspondances fortes.


In [ ]:
# Ajout des clusters au DataFrame original
data_with_clusters = data.merge(
    data_songs[['track_artist', 'track_name', 'cluster']],
    on=['track_artist', 'track_name'],
    how='left'  
)

cluster_genre_table = pd.crosstab(
    data_with_clusters['playlist_genre'],
    data_with_clusters['cluster'],
    normalize='index'
)

sns.heatmap(cluster_genre_table, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Proportions des genres par cluster")
plt.xlabel("Cluster")
plt.ylabel("genre")
plt.show()

In [ ]:
# crosstab cluster vs subgenre
cluster_subgenre_table = pd.crosstab(
    data_with_clusters['playlist_subgenre'],
    data_with_clusters['cluster'],

    normalize='index'
)
sns.heatmap(cluster_subgenre_table, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("\nProportions des sous-genres par cluster")
plt.xlabel("Cluster")
plt.ylabel("Sous-genre")
plt.show()

Les résultats des crosstabs confirment nos analyses précédentes :

* **Cluster 0** est fortement associé aux genres **rock** et sous-genres comme **hard rock, album rock, ou progressive electro house**, ce qui correspond à son profil dynamique, énergique et peu acoustique.

* **Cluster 1** regroupe une part importante de morceaux **r&b, rap,** et des sous-genres comme **trap, hip pop, ou neo soul**, en cohérence avec son profil très dansant, populaire et modérément énergique.

* **Cluster 2** est plus diffus mais montre une surreprésentation de sous-genres comme **hip hop ou neo soul**, ce qui correspond à un profil plus calme, acoustique ou instrumental.

* **Cluster 3** concentre une forte proportion de morceaux **latin** et des sous-genres comme **reggaeton, latin pop, ou dance pop**, ce qui colle avec son profil festif, très dansant et joyeux. 

* **Cluster 4** est particulièrement présent dans des sous-genres instrumentaux ou **électro comme electro house, progressive electro house, ou big room**, ce qui correspond à son profil instrumental, dynamique et peu populaire.

Ainsi, la correspondance entre clusters et genres/sous-genres musicaux, mise en évidence par les heatmaps, valide la pertinence et la cohérence de notre segmentation : chaque cluster regroupe bien des morceaux aux caractéristiques musicales proches, confirmant les profils identifiés par l’analyse descriptive.

## Modèle de Mélange Gaussien (GMM)

Le modèle de mélange gaussien (GMM) est une méthode de clustering probabiliste qui permet de modéliser les données comme un mélange de plusieurs distributions gaussiennes. Contrairement à K-Means, GMM peut capturer des formes de clusters plus complexes et des variances différentes entre les clusters.

Nous allons appliquer le GMM sur les mêmes données que pour K-Means, en utilisant les trois premières composantes principales de l'ACP. Nous allons également utiliser le critère de BIC pour déterminer le nombre optimal de clusters.

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [ ]:
bic_scores = []
n_components_range = range(2, 13)
X_pca = pca_songs_df[['CP1', 'CP2', 'CP3']].copy()

# Calcul des scores pour chaque nombre de clusters
for n in n_components_range:
    gmm = GaussianMixture(n_components=n, random_state=42)
    gmm.fit(X_pca)
    labels = gmm.predict(X_pca)
    
    bic_scores.append(gmm.bic(X_pca))
    
scores_df = pd.DataFrame({
    "Nombre de clusters": n_components_range,
    "BIC": bic_scores,
})

plt.figure(figsize=(12, 6))
sns.set(style="whitegrid")

sns.lineplot(data=scores_df, x="Nombre de clusters", y="BIC", marker="o", label="BIC", color="blue")

bic_min = scores_df.loc[scores_df["BIC"].idxmin(), "Nombre de clusters"]

plt.axvline(x=bic_min, color="blue", linestyle="--", alpha=0.5)
plt.title("Évaluation du nombre de clusters GMM (BIC)")
plt.xlabel("Nombre de clusters")
plt.ylabel("Score")
plt.legend()
plt.tight_layout()
plt.show()

scores_df['BIC_diff'] = scores_df['BIC'].diff().abs()
# Affichage du plus grand saut de score
max_bic_diff = scores_df['BIC_diff'].max()
max_bic_diff_index = scores_df['BIC_diff'].idxmax()
if max_bic_diff_index > 0:
    print(f"\nMaximum BIC difference is between {scores_df['Nombre de clusters'][max_bic_diff_index-1]} and {scores_df['Nombre de clusters'][max_bic_diff_index]} clusters: {max_bic_diff}")
else:
    print("\nNo BIC difference to display.")

Ici, le critère BIC indique que le nombre optimal de clusters est 10 clusters. Ce choix optimise le critère statistique, mais implique des groupes plus petits et une interprétation potentiellement plus complexe.

Nous avons donc décidé de retenir ce nombre de clusters pour la suite de l’analyse, sans chercher à simplifier davantage le modèle. Les analyses et visualisations suivantes seront donc réalisées avec 10 clusters, conformément à l’optimum donné par le BIC.

In [ ]:
# Appliquer le modèle GMM et obtenir les labels
gmm = GaussianMixture(n_components=10, random_state=42)
gmm.fit(pca_songs_df[['CP1', 'CP2', 'CP3']])
labels_gmm = gmm.predict(pca_songs_df[['CP1', 'CP2', 'CP3']])

# Ajouter les clusters GMM au DataFrame
pca_songs_df['cluster_gmm'] = labels_gmm
data_songs['cluster_gmm'] = labels_gmm

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.scatterplot(x=pca_songs_df['CP1'], y=pca_songs_df['CP2'], hue=labels_10, palette='tab20', s=10, alpha=0.7, ax=axes[0])
axes[0].set_title('GMM - 10 clusters : CP1 vs CP2')
axes[0].axhline(0, linestyle='--', color='gray', linewidth=0.5)
axes[0].axvline(0, linestyle='--', color='gray', linewidth=0.5)
axes[0].legend(title='Cluster', loc='best', fontsize='small')


sns.scatterplot(x=pca_songs_df['CP2'], y=pca_songs_df['CP3'], hue=labels_10, palette='tab10', s=10, alpha=0.7, ax=axes[1])
axes[1].set_title('GMM - 10 clusters : CP2 vs CP3')
axes[1].axhline(0, linestyle='--', color='gray', linewidth=0.5)
axes[1].axvline(0, linestyle='--', color='gray', linewidth=0.5)
axes[1].legend(title='Cluster', loc='best', fontsize='small')

sns.scatterplot(x=pca_songs_df['CP1'], y=pca_songs_df['CP3'], hue=labels_10, palette='tab20', s=10, alpha=0.7, ax=axes[2])
axes[2].set_title('GMM - 10 clusters : CP1 vs CP3')
axes[2].axhline(0, linestyle='--', color='gray', linewidth=0.5)
axes[2].axvline(0, linestyle='--', color='gray', linewidth=0.5)
axes[2].legend(title='Cluster', loc='best', fontsize='small')

plt.tight_layout()
plt.show()


D'après nos graphes, dans certains plans les classes se distinguent assez, mais dans d'autres plans on peut voir des superpositions sans affichage distinct (CP2 vs. CP3 et CP1 vs. CP3). 

Nous allons maintenant afficher les boxplots, afin de trouver des profils type dans nos clusters. 

In [ ]:
data_songs['cluster_gmm'] = pca_songs_df['cluster_gmm'].values
n_cols = 4
n_rows = (len(features) + n_cols - 1) // n_cols  

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.boxplot(x='cluster_gmm', y=feature, data=data_songs, ax=axes[i], showfliers=False)
    axes[i].set_title(f'{feature} par cluster GMM')

for j in range(len(features), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# Merge des clusters GMM dans data initial
data_with_gmm_clusters = data.merge(
    data_songs[['track_artist', 'track_name', 'cluster_gmm']],
    on=['track_artist', 'track_name'],
    how='left'
)

# Crosstab genre vs cluster_gmm
cluster_gmm_genre_table = pd.crosstab(
    data_with_gmm_clusters['playlist_genre'],
    data_with_gmm_clusters['cluster_gmm'],
    normalize='index'
)

# Crosstab subgenre vs cluster_gmm
cluster_gmm_subgenre_table = pd.crosstab(
    data_with_gmm_clusters['playlist_subgenre'],
    data_with_gmm_clusters['cluster_gmm'],
    normalize='index'
)

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.heatmap(cluster_gmm_genre_table, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[0])
axes[0].set_title("Répartition des genres par cluster GMM")
axes[0].set_xlabel("Cluster")
axes[0].set_ylabel("Genre")

sns.heatmap(cluster_gmm_subgenre_table, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[1])
axes[1].set_title("Répartition des sous-genres par cluster GMM")
axes[1].set_xlabel("Cluster")
axes[1].set_ylabel("Sous-genre")

plt.tight_layout()
plt.show()

Nous effectuons une analyse croisée directement entre les profils des clusters obtenu par GMM et la répartition des genres et sous-genres :

**Cluster 0 :**  
- **Profil audio** : Très dynamique, dansant, joyeux, peu acoustique.
- **Genres dominants** : EDM (22 %), pop (22 %), latin (19 %), rock (13 %), rap (13 %).
- **Sous-genres marquants** : dance pop (30 %), big room (22 %), electro house (22 %), pop edm (25 %), post-teen pop (26 %).
- **Conclusion** : Ce cluster regroupe surtout des morceaux électro, dance et pop très énergiques.

**Cluster 1 :**  
- **Profil audio** : Dansant, modérément dynamique, populaire.
- **Genres dominants** : pop (22 %), rap (19 %), r&b (17 %), latin (17 %), rock (14 %), edm (11 %).
- **Sous-genres marquants** : dance pop (25 %), hip pop (24 %), electropop (19 %), classic rock (17 %), indie poptimism (22 %), trap (27 %), tropical (27 %).
- **Conclusion** : Cluster très varié, mais avec une forte composante pop urbaine et dance.

**Cluster 2 :**  
- **Profil audio** : Calme, très acoustique et instrumental, peu joyeux.
- **Genres dominants** : pop (5 %), r&b (9 %), rock (6 %), edm (1 %), latin (3 %), rap (6 %).
- **Sous-genres marquants** : hip hop (20 %), neo soul (12 %), instrumental, classic rock (6 %).
- **Conclusion** : Morceaux calmes, ambiance, ballades, parfois hip-hop ou soul.

**Cluster 3 :**  
- **Profil audio** : Très instrumental, long, peu populaire.
- **Genres dominants** : edm (9 %), rock (3 %), pop (2 %), latin (2 %), r&b (1 %), rap (0.5 %).
- **Sous-genres marquants** : progressive electro house (19 %), electro house (11 %), big room (3 %).
- **Conclusion** : Cluster très spécifique, morceaux instrumentaux, électro ou rock progressif.

**Cluster 4 :**  
- **Profil audio** : Dynamique, rapide, peu instrumental, ambiance électro/rock.
- **Genres dominants** : rock (24 %), edm (19 %), pop (11 %), rap (5 %), r&b (3 %), latin (3 %).
- **Sous-genres marquants** : hard rock (40 %), big room (30 %), permanent wave (23 %), pop edm (17 %).
- **Conclusion** : Morceaux rock énergique, électro rapide, hard rock.

**Cluster 5 :**  
- **Profil audio** : Puissant, instrumental, long, peu populaire.
- **Genres dominants** : edm (20 %), rock (10 %), pop (6 %), latin (2 %), r&b (2 %), rap (3 %).
- **Sous-genres marquants** : big room (31 %), progressive electro house (25 %), electro house (18 %).
- **Conclusion** : Morceaux électro instrumentaux, rock progressif, BO.

**Cluster 6 :**  
- **Profil audio** : Très dansant, joyeux, populaire, peu acoustique.
- **Genres dominants** : latin (30 %), rap (26 %), r&b (15 %), pop (12 %), edm (6 %), rock (4 %).
- **Sous-genres marquants** : reggaeton (50 %), latin pop (31 %), latin hip hop (27 %), gangster rap (31 %), hip hop (28 %), hip pop (25 %).
- **Conclusion** : Morceaux très populaires, ambiance reggaeton, latin, hip-hop.

**Cluster 7 :**  
- **Profil audio** : Calme, acoustique, indie/alternative.
- **Genres dominants** : pop (8 %), r&b (12 %), rock (13 %), edm (4 %), latin (5 %), rap (4 %).
- **Sous-genres marquants** : album rock (18 %), classic rock (15 %), neo soul (15 %), new jack swing (7 %).
- **Conclusion** : Indie, alternative, soul, rock doux.

**Cluster 8 :**  
- **Profil audio** : Très dansant, joyeux, pop/dance.
- **Genres dominants** : latin (8 %), r&b (19 %), pop (7 %), edm (2 %), rock (7 %), rap (13 %).
- **Sous-genres marquants** : neo soul (25 %), hip hop (15 %), hip pop (15 %), new jack swing (17 %).
- **Conclusion** : Dance, pop joyeuse, soul, hip-hop.

**Cluster 9 :**  
- **Profil audio** : Très dynamique, joyeux, long, festif.
- **Genres dominants** : r&b (16 %), latin (11 %), pop (6 %), edm (7 %), rock (6 %), rap (10 %).
- **Sous-genres marquants** : new jack swing (44 %), latin hip hop (18 %), neo soul (14 %).
- **Conclusion** : Morceaux festifs, dance longue, influence soul/latin.


**Conclusion** :  
Les heatmaps confirment et précisent les profils audio : chaque cluster GMM regroupe des morceaux aux caractéristiques musicales proches, avec des genres et sous-genres dominants cohérents avec les profils identifiés par les statistiques descriptives.  
Cela valide la pertinence de la segmentation et facilite l’interprétation des groupes.

## Classification hiérarchique ascendante (CAH)

La classification ascendante hiérarchique (CAH) est une méthode de clustering qui permet de regrouper les données en formant une hiérarchie de clusters. Contrairement à K-Means ou GMM, la CAH ne nécessite pas de spécifier le nombre de clusters à l’avance, mais produit un dendrogramme qui permet de visualiser la structure des données et de choisir un niveau de coupe pour obtenir un nombre souhaité de clusters.

Nous allons appliquer la CAH sur les mêmes données que pour K-Means et GMM, en utilisant les trois premières composantes principales de l'ACP. Nous allons ensuite visualiser le dendrogramme pour déterminer le nombre de clusters à retenir.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.metrics import silhouette_score

Tout d'abord, comme pour les méthodes précédentes, il faut trouver le nombre de clusters optimal. Pour cela nous allons utiliser deux méthodes : la méthode du score silhouette, qui va nous renvoyer le nombre de clusters pour lequel le score est maximal, et une méthode visuelle à l'aide d'un dendrogramme.   

**Score silhouette**

In [ ]:
Z = linkage(X, method='ward')
scores = []
for k in range(2, 8):
    labels = fcluster(Z, k, criterion='maxclust')
    score = silhouette_score(X, labels)
    scores.append(score)
optimal_k = range(2, 8)[scores.index(max(scores))]
print("Nombre optimal de clusters (CAH, silhouette):", optimal_k)

**Dendrogramme**

In [ ]:
plt.figure(figsize=(12, 8))

Z = linkage(pca_songs_df[['CP1', 'CP2', 'CP3']], method='ward')
dendrogram(Z, leaf_rotation=90, leaf_font_size=12, no_labels=True)

plt.axhline(y=170, color='red', linestyle='--')
plt.title('Dendrogramme - Méthode Ward')
plt.xlabel('Index des morceaux (non affichés)')
plt.ylabel('Distance')
plt.show()

Nous pouvons constater que les deux méthodes ne donnent pas le même nombre de clusters optimaux. En effet, le **score silhouette est maximal pour 2 clusters** tandis que le **dendrogramme est optimal pour 3 clusters**. 
 
Nous allons projeter les clusters dans les plans factoriels afin de les comparer. 

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# CAH avec 2 clusters 
n_clusters = 2
labels_cah_2 = fcluster(Z, n_clusters, criterion='maxclust')
unique_labels_2 = np.sort(np.unique(labels_cah_2))
cah_label_map_2 = {old: new for new, old in enumerate(unique_labels_2)}
labels_cah_remapped_2 = np.vectorize(cah_label_map_2.get)(labels_cah_2)

for i, (x, y) in enumerate(combinations):
    sns.scatterplot(
        data=pca_songs_df.assign(cluster_cah_2=labels_cah_remapped_2),
        x=x,
        y=y,
        hue='cluster_cah_2',
        palette='Set2',
        s=10,
        alpha=0.7,
        ax=axes[0, i]
    )
    axes[0, i].set_title(f'CAH - 2 clusters : {x} vs {y}')
    axes[0, i].axhline(0, linestyle='--', color='gray', linewidth=0.5)
    axes[0, i].axvline(0, linestyle='--', color='gray', linewidth=0.5)
    axes[0, i].legend(title='Cluster', loc='best')

# CAH avec 3 clusters (retenu pour la suite)
n_clusters = 3
labels_cah_3 = fcluster(Z, n_clusters, criterion='maxclust')
unique_labels_3 = np.sort(np.unique(labels_cah_3))
cah_label_map_3 = {old: new for new, old in enumerate(unique_labels_3)}
labels_cah_remapped_3 = np.vectorize(cah_label_map_3.get)(labels_cah_3)

pca_songs_df['cluster_cah_3'] = labels_cah_remapped_3
data_songs['cluster_cah_3'] = labels_cah_remapped_3

for i, (x, y) in enumerate(combinations):
    sns.scatterplot(
        data=pca_songs_df,
        x=x,
        y=y,
        hue='cluster_cah_3',
        palette='Set2',
        s=10,
        alpha=0.7,
        ax=axes[1, i]
    )
    axes[1, i].set_title(f'CAH - 3 clusters : {x} vs {y}')
    axes[1, i].axhline(0, linestyle='--', color='gray', linewidth=0.5)
    axes[1, i].axvline(0, linestyle='--', color='gray', linewidth=0.5)
    axes[1, i].legend(title='Cluster', loc='best')

plt.tight_layout()
plt.show()

**Projection de 2 clusters**  
On observe une séparation assez grossière de l'ensemble des morceaux en deux groupes. Les clusters ne sont pas de taille homogène, mettant en évidence l'opposition structurelle dans les données, mais chaque cluster regroupe une grande diversité de morceaux, donc cela pourrait se traduire par une forte hétérogénéité dans les analyses futures. 

**Projection de 3 clusters**  
Avec trois clusters, la séparation devient automatiquement plus fine. 
* Dans certains plans factoriels la distinction des classes est assez simple, et dans d'autres plans les classes peuvent être légèrement mélangées. 
* Cette segmentation permet de mieux capturer la diversité des profils musicaux. **C'est celle que nous allons conserver dans les analyses suivantes**.

In [ ]:
data_songs['cluster_cah_3'] = pca_songs_df['cluster_cah_3'].values

data_with_cah_clusters = data.merge(
    data_songs[['track_artist', 'track_name', 'cluster_cah_3']],
    on=['track_artist', 'track_name'],
    how='left'
)

In [ ]:
n_cols = 4
n_rows = (len(features) + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
axes = axes.flatten()

for i, feature in enumerate(features):
    sns.boxplot(x='cluster_cah_3', y=feature, data=data_songs, ax=axes[i], showfliers=False, width=0.4)
    axes[i].set_title(f'{feature} par cluster CAH')

for j in range(len(features), len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
# Crosstab genre vs cluster_cah_3
cluster_cah_genre_table = pd.crosstab(
    data_with_cah_clusters['playlist_genre'],
    data_with_cah_clusters['cluster_cah_3'],
    normalize='index'
)

# Crosstab subgenre vs cluster_cah_3
cluster_cah_subgenre_table = pd.crosstab(
    data_with_cah_clusters['playlist_subgenre'],
    data_with_cah_clusters['cluster_cah_3'],
    normalize='index'
)

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.heatmap(cluster_cah_genre_table, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[0])
axes[0].set_title("Répartition des genres par cluster CAH")
axes[0].set_xlabel("Cluster")
axes[0].set_ylabel("Genre")

sns.heatmap(cluster_cah_subgenre_table, annot=True, cmap='coolwarm', fmt='.2f', ax=axes[1])
axes[1].set_title("Répartition des sous-genres par cluster CAH")
axes[1].set_xlabel("Cluster")
axes[1].set_ylabel("Sous-genre")

plt.tight_layout()
plt.show()

Comme pour les méthodes précédentes, nous croisons nos analyses avec les genres des playlists, nous permettant de mettre en avant des profils audios. 

**Cluster 0**
- **Profil audio** :  
  - *Danceability* moyenne, *energy* faible, *loudness* faible
  - Très *acoustique* (0.56), *instrumentalness* élevé, *valence* faible
  - *Popularité* la plus haute
- **Genres dominants** :  
  - Surreprésentation de **r&b** (23 %), **rock** (13 %), **pop** (11 %)
- **Sous-genres marquants** :  
  - *neo soul*, *hip hop*, *indie poptimism*, *classic rock*, *album rock*
- **Interprétation** : Morceaux calmes, acoustiques, parfois instrumentaux, souvent issus du r&b, rock doux ou indie.

**Cluster 1**
- **Profil audio** :  
  - *Danceability* très élevée, *energy* moyenne-haute, *loudness* modéré 
  - *Acousticness* faible, *instrumentalness* très faible, *valence* très élevée
  - *Popularité* moyenne
- **Genres dominants** :  
  - **latin** (62 %), **rap** (61 %), **r&b** (53 %), **pop** (38 %)
- **Sous-genres marquants** :  
  - *reggaeton*, *latin hip hop*, *new jack swing*, *dance pop*, *hip pop*, *trap*
- **Interprétation** : Morceaux très dansants, festifs, joyeux, peu acoustiques, typiques du latin, rap, r&b moderne.

**Cluster 2**
- **Profil audio** :  
  - *Danceability* moyenne-basse, *energy* la plus élevée, *loudness* la plus forte
  - *Acousticness* très faible, *instrumentalness* la plus élevée, *valence* moyenne
  - *Popularité* la plus faible
- **Genres dominants** :  
  - **edm** (73 %), **rock** (66 %), **pop** (51 %)
- **Sous-genres marquants** :  
  - *big room*, *progressive electro house*, *hard rock*, *electro house*, *permanent wave*
- **Interprétation** : Morceaux très dynamiques, puissants, peu acoustiques, souvent instrumentaux, typiques de l’EDM, rock énergique ou électro.


**Conclusion**  
La CAH distingue :
- **Cluster 0** : Calme, acoustique, r&b/rock/indie, populaire.
- **Cluster 1** : Très dansant, festif, latin/rap/r&b, joyeux, populaire.
- **Cluster 2** : Très dynamique, EDM/rock, instrumental, peu populaire, morceaux longs.

-----------------------------------------------------------------------------

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from scipy.optimize import linear_sum_assignment

data_with_clusters = data.merge(
    data_songs[['track_artist', 'track_name', 'cluster', 'cluster_gmm', 'cluster_cah_3']],
    on=['track_artist', 'track_name'],
    how='left'
)
# --- Comparaison CAH vs GMM ---
cm_cah_gmm = confusion_matrix(data_with_clusters['cluster_cah_3'], data_with_clusters['cluster_gmm'])
row_ind_cah_gmm, col_ind_cah_gmm = linear_sum_assignment(-cm_cah_gmm)
mapping_cah_gmm = {old: new for old, new in zip(col_ind_cah_gmm, row_ind_cah_gmm)}
data_with_clusters['cluster_gmm_aligned_to_cah'] = data_with_clusters['cluster_gmm'].map(mapping_cah_gmm)
cm_cah_gmm_aligned = confusion_matrix(
    data_with_clusters['cluster_cah_3'],
    data_with_clusters['cluster_gmm_aligned_to_cah']
)
ConfusionMatrixDisplay(cm_cah_gmm_aligned).plot()
plt.xlabel('Clusters GMM (alignés sur CAH)')
plt.ylabel('Clusters CAH')
plt.title('Comparaison des clusters CAH et GMM (alignés)')
plt.show()

# --- Comparaison CAH vs KMeans ---
cm_cah_kmeans = confusion_matrix(data_with_clusters['cluster_cah_3'], data_with_clusters['cluster'])
row_ind_cah_kmeans, col_ind_cah_kmeans = linear_sum_assignment(-cm_cah_kmeans)
mapping_cah_kmeans = {old: new for old, new in zip(col_ind_cah_kmeans, row_ind_cah_kmeans)}
data_with_clusters['cluster_kmeans_aligned_to_cah'] = data_with_clusters['cluster'].map(mapping_cah_kmeans)
cm_cah_kmeans_aligned = confusion_matrix(
    data_with_clusters['cluster_cah_3'],
    data_with_clusters['cluster_kmeans_aligned_to_cah']
)
ConfusionMatrixDisplay(cm_cah_kmeans_aligned).plot()
plt.xlabel('Clusters KMeans (alignés sur CAH)')
plt.ylabel('Clusters CAH')
plt.title('Comparaison des clusters CAH et KMeans (alignés)')
plt.show()

## Comparaison des méthodes de clustering
Dans cette partie, nous appliquons une Analyse des Correspondances Multiples (MCA) pour comparer les résultats des différentes méthodes de clustering (K-Means, GMM, CAH). 
L’objectif est de visualiser les clusters obtenus par chaque méthode dans un espace factoriel commun, afin d’évaluer leur cohérence et leur séparation.

In [ ]:
import prince

# On réduit certaines modalités afin de rendre l'analyse plus simple.
# La MCA peut-être sensible aux nombre de modalités dans les colonnes catégorielles.
freq_artists = data['track_artist'].value_counts()
top_artists = freq_artists[freq_artists > 60].index
data['track_artist_reduced'] = data['track_artist'].apply(lambda x: x if x in top_artists else 'Other')

freq_albums = data['track_album_name'].value_counts()
top_albums = freq_albums[freq_albums > 10].index
data['track_album_name_reduced'] = data['track_album_name'].apply(lambda x: x if x in top_albums else 'Other')

# Variables catégorielles sélectionnées pour la MCA
cols_cat_reduced = ['track_artist_reduced', 'track_album_name_reduced', 'playlist_name', 
                    'playlist_genre', 'playlist_subgenre', 'key', 'mode']


# Echantillonnage pour pour éviter les problèmes de performance
# Vérification des valeurs manquantes
data_sample = data.sample(n=5000, random_state=42)
data_sample_clean = data_sample[cols_cat_reduced].astype(str)

mca = prince.MCA(n_components=2, n_iter=3, random_state=42)
mca_result = mca.fit_transform(data_sample_clean)

eig_vals = mca.percentage_of_variance_
cumul_vals = mca.cumulative_percentage_of_variance_

On applique maintenant nos méthodes de clustering. 

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

X = mca_result.values  # coordonnées MCA

# KMeans avec 5 clusters
kmeans = KMeans(n_clusters=5, random_state=42)
labels_kmeans = kmeans.fit_predict(X)

# GMM avec 10 clusters
gmm = GaussianMixture(n_components=10, random_state=42)
labels_gmm = gmm.fit_predict(X)

# CAH avec 3 clusters
Z = linkage(X, method='ward')
labels_cah = fcluster(Z, t=3, criterion='maxclust')

Nous allons maintenant afficher dans le plan de notre MCA nos clusters qui ont été obtenus. Afin de comparer les méthodes, nous allons afficher différents scores. 

* Le **silhouette score**, qu'on veut le plus élevé possible. Ce score mesure la cohésion interne des clusters et leur séparation.
* Le score de **Calinski-Harabasz**, plus il est élevé, mieux c'est. Il mesure la séparation des clusters et leur compacité. 
* Le score de **Davies-Bouldin**, contrairement aux autres mesures, plus le score est bas, mieux c'est. Ce score mesure la similarité entre les clusters, et nous souhaitons des clusters différents. 

In [ ]:
# Correction pour les pourcentages de variance expliquée dans la MCA

# Récupérer les pourcentages de variance expliquée de la MCA
explained_var_mca = [eig_vals[0], eig_vals[1]]  # ou utiliser mca.eigenvalues_summary si disponible

scores = {
    'Méthode': ['KMeans (5)', 'GMM (10)', 'CAH (3)'],
    'Silhouette': [
        silhouette_score(X, labels_kmeans),
        silhouette_score(X, labels_gmm),
        silhouette_score(X, labels_cah)
    ],
    'Calinski-Harabasz': [
        calinski_harabasz_score(X, labels_kmeans),
        calinski_harabasz_score(X, labels_gmm),
        calinski_harabasz_score(X, labels_cah)
    ],
    'Davies-Bouldin': [
        davies_bouldin_score(X, labels_kmeans),
        davies_bouldin_score(X, labels_gmm),
        davies_bouldin_score(X, labels_cah)
    ]
}
df_scores = pd.DataFrame(scores)

fig, axs = plt.subplots(1, 4, figsize=(22,5))

# Plot KMeans
axs[0].scatter(X[:,0], X[:,1], c=labels_kmeans, cmap='Set2', s=10)
axs[0].set_title('KMeans (5 clusters)')
axs[0].set_xlabel('MCA Dim 1')
axs[0].set_ylabel('MCA Dim 2')

# Plot GMM
axs[1].scatter(X[:,0], X[:,1], c=labels_gmm, cmap='Set2', s=10)
axs[1].set_title('GMM (10 clusters)')
axs[1].set_xlabel('MCA Dim 1')

# Plot CAH
axs[2].scatter(X[:,0], X[:,1], c=labels_cah, cmap='Set2', s=10)
axs[2].set_title('CAH (3 clusters)')
axs[2].set_xlabel('MCA Dim 1')

# Affichage des scores
axs[3].axis('off')
table_text = df_scores.round(3).to_string(index=False)
axs[3].text(0, 0.5, table_text, fontsize=12, fontfamily='monospace')

plt.tight_layout()
plt.show()

D'après nos graphiques, nous pouvons juger la séparation spatiale des clusters. 

Les plots obtenus par K-Means et CAH semblent bien séparer chaque classe. Pour celui obtenu par GMM, nous pouvons voir qu'il y a des recouvrements entre les classes, ce qui indique une méthode plus faible pour les séparations de classes. 

Les scores nous fournissent les informations suivantes :
* **Silhouette :** 
  * **K-Means** a de loin le meilleur score (0.73), indiquant que les clusters sont très bien séparés. 
  * GMM et CAH ont eux des scores moins bons (0.58 et 0.63 respectivement).
* **Calinski-Harabasz :**
  * De nouveau **K-Means** possède le score le plus élevé, ce qui confirme la qualité de séparation.
  * De même, GMM et CAH sont nettement en dessous. 
* **Davies-Bouldin :**
  * **K-Means** a le score le plus bas (0.34), donc les classes sont distinctes. 
  * GMM a un score de 1.17, ce qui est très élevé. Cela confirme notre analyse visuelle. 
  * CAH lui a un score moyen. 

**Conclusion :** K-Means est de loin la méthode la plus adaptée à notre jeu de données. GMM semble sur-segmenter les données, et CAH forme des clusters plus larges, donc moins séparés. 

Nous affichons maintenant la répartition des clusters ainsi que les genres et sous-genres dans l'espace MCA. 

In [ ]:
# Coordonnées des modalités (genres et sous-genres)
mod_coords = mca.column_coordinates(data_sample_clean)

# Ajouter les clusters KMeans au DataFrame data_sample
data_sample['cluster_kmeans'] = labels_kmeans

# Barycentres des clusters dans l'espace MCA
cluster_centers = []
for k in sorted(data_sample['cluster_kmeans'].unique()):
    cluster_centers.append(X[data_sample['cluster_kmeans'] == k].mean(axis=0))
cluster_centers = np.array(cluster_centers)

plt.figure(figsize=(12, 10))

import matplotlib.pyplot as plt

# Coordonnées des modalités
mod_coords = mca.column_coordinates(data_sample_clean)

plt.figure(figsize=(12, 10))

# Projection des modalités de genre
for idx in mod_coords.index:
    if idx.startswith('playlist_genre_'):
        plt.scatter(mod_coords.loc[idx, 0], mod_coords.loc[idx, 1], color='red', marker='x', s=80, label='Genre' if idx == 'playlist_genre_' + data_sample_clean['playlist_genre'].unique()[0] else "")
        plt.text(mod_coords.loc[idx, 0], mod_coords.loc[idx, 1], idx.replace('playlist_genre_', ''), color='red', fontsize=12)


# Projection des sous-genres
for idx in mod_coords.index:
    if idx.startswith('playlist_subgenre_'):
        plt.scatter(mod_coords.loc[idx, 0], mod_coords.loc[idx, 1], color='green', marker='^', s=50, label='Sous-genre' if idx == 'playlist_subgenre_' + data_sample_clean['playlist_subgenre'].unique()[0] else "")
        plt.text(mod_coords.loc[idx, 0], mod_coords.loc[idx, 1], idx.replace('playlist_subgenre_', ''), color='green', fontsize=8)

plt.scatter(cluster_centers[:,0], cluster_centers[:,1], color='blue', s=100, label='Clusters')
for i, (x, y) in enumerate(cluster_centers):
    plt.text(x, y, f'Cluster {i}', color='blue', fontsize=12, fontweight='bold')

plt.xlabel('MCA Dim 1')
plt.ylabel('MCA Dim 2')
plt.title("Projection MCA : Genres, Sous-genres et barycentres des clusters")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**Clusters et barycentres**

Les barycentres des clusters (points bleus) indiquent la position moyenne des morceaux appartenant à chaque cluster dans l'espace MCA.
Les clusters semblent bien séparés dans l'espace, ce qui suggère une bonne différenciation entre les groupes identifiés par l'algorithme de clustering.

**Genres (croix rouges)**

Les genres principaux **(pop, rap, latin, rock, etc.)** sont représentés par des croix rouges.

Chaque genre est positionné dans l'espace en fonction des caractéristiques qualitatives associées aux morceaux qui lui appartiennent.

Par exemple :
* **pop** est situé dans une région proche des sous-genres comme **dance pop** et **indie poptimism**, ce qui reflète des caractéristiques communes.
* **latin** est proche de sous-genres comme **reggaeton, latin pop, et tropical**, ce qui est cohérent avec son profil festif et dansant.
* **rock** est associé à des sous-genres comme **hard rock, classic rock, et album rock,** soulignant son caractère acoustique et énergique.

**Sous-genres (triangles verts)**

Les sous-genres sont représentés par des triangles verts et sont regroupés autour des genres auxquels ils appartiennent.
Par exemple :
* Les sous-genres **trap, hip hop, et gangster rap** sont proches du genre **rap**, ce qui reflète leur forte association.
* Les sous-genres progressive **electro house, electro house, et big room** sont proches de **edm**, ce qui correspond à leur nature électronique et instrumentale.

**Interprétation des clusters**

**Cluster 0 :** Situé dans une région associée à des sous-genres comme **electropop, post-teen pop, et dance pop,** ce cluster semble regrouper des morceaux pop modernes et dansants.

**Cluster 1 :** Proche de **latin, reggaeton, et latin pop**, ce cluster regroupe des morceaux festifs et dansants, typiques de la musique latine.

**Cluster 2 :** Associé à **rock, hard rock, et classic rock**, ce cluster regroupe des morceaux dynamiques et acoustiques.

**Cluster 3 :** Proche de sous-genres électroniques comme **progressive electro house et electro house,** ce cluster regroupe des morceaux instrumentaux et dynamiques.

**Cluster 4 :** Isolé dans une région éloignée, ce cluster semble regrouper des morceaux très spécifiques, peut-être instrumentaux ou expérimentaux.

**Séparation des clusters**

Les clusters sont bien séparés dans l'espace MCA, ce qui indique une bonne cohérence des groupes formés.
Les genres et sous-genres proches des barycentres des clusters confirment la pertinence des regroupements.

**Conclusion**

Ce graphique met en évidence une bonne correspondance entre les clusters identifiés et les genres/sous-genres musicaux. Les barycentres des clusters sont cohérents avec les genres et sous-genres associés, ce qui valide la segmentation réalisée.

## Système de recommandation

### Objectif
L'objectif de cette section est de développer un système de recommandation basé sur les profils musicaux latents identifiés par la NMF. Ce système permettra de :

1. Recommander des **morceaux à ajouter à une playlist existante** en analysant son profil musical global
2. Suggérer des **chansons similaires à un morceau spécifique choisi** par l'utilisateur

### Méthodologie
En se basant sur les 6 profils latents extraits (instrumental, énergique-intense, live, émotionnel-valence, acoustique-chant, rap-rythmé), le système calcule des mesures de similarité pour proposer des recommandations personnalisées et pertinentes.

1. **Extraction des profils latents** : Utilisation des 6 profils musicaux latents identifiés par la NMF.
2. **Calcul de similarité** : Mesures de similarité entre les morceaux ou le profil moyen d'une playlist et les profils latents pour déterminer les recommandations.
3. **Recommandation personnalisée** : Proposer des morceaux en fonction des similarités calculées.

Nous commençons par effectuer une recommandation à partir d'une chanson spécfique. Pour cela, nous allons utiliser la fonction `recommend_similar_tracks` qui prend en entrée un morceau et renvoie les morceaux similaires en fonction de leur profil musical. 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def recommend_similar_tracks(track_name, track_artist, num_recommendation=3):
    # Trouver l'index de la chanson dans data_songs
    song_idx = data_songs[
        (data_songs['track_name'] == track_name) & 
        (data_songs['track_artist'] == track_artist)
    ].index
    
    if len(song_idx) == 0:
        return f"Track '{track_name}' by '{track_artist}' not found in the dataset."
    
    song_idx = song_idx[0]
    
    # Récupérer le profil de la chanson dans profile_weights
    song_profile = profile_weights.loc[song_idx].values.reshape(1, -1)
    
    # Calculer les similarités cosinus avec toutes les autres chansons
    similarities = cosine_similarity(song_profile, profile_weights).flatten()
    
    # Trier les indices des chansons par similarité décroissante
    similar_indices = similarities.argsort()[::-1]
    
    # Exclure la chanson elle-même et prendre les num_recommendation plus proches
    similar_indices = [idx for idx in similar_indices if idx != song_idx][1:num_recommendation+1]
    
    # Récupérer les informations des chansons similaires
    similar_tracks = data_songs.iloc[similar_indices][['track_name', 'track_artist']]
    similar_tracks['similarity'] = similarities[similar_indices]
    
    return similar_tracks.reset_index(drop=True)

# Exemple d'utilisation
recommend_similar_tracks("Thunderstruck", "AC/DC", num_recommendation=3)

L'algorithme nous renvoie les morceaux les plus similaires au morceau choisi, en se basant sur les profils musicaux latents. Ce sont des morceaux qui partagent des caractéristiques audio avec `Thunderstruck`.

Maintenant nous allons faire une recommandation de morceaux à ajouter à une playlist existante avec la fonction `recommend_similar_tracks_playlist` qui prend en entrée une playlist et renvoie les morceaux similaires en fonction du profil musical moyen de la playlist.

A partir de la playlist Dance Pop: Japan, nous allons extraire le profil musical moyen de cette playlist et recommander des morceaux similaires en fonction des profils musicaux latents. 

In [ ]:
data_songs_with_playlist = data_songs.copy()
data_songs_with_playlist['playlist_name'] = data['playlist_name']


def recommend_similar_tracks_playlist(playlist_name, num_recommendation=3):
    # faire la moyenne des profils des chassons de la playlist
    playlist_tracks = data_songs_with_playlist[data_songs_with_playlist['playlist_name'] == playlist_name]
    if playlist_tracks.empty:
        return f"Playlist '{playlist_name}' not found in the dataset."
    
    # Calculer le profil moyen de la playlist
    playlist_profile = profile_weights.loc[playlist_tracks.index].mean().values.reshape(1, -1)

    # Calculer les similarités cosinus avec toutes les autres chansons
    similarities = cosine_similarity(playlist_profile, profile_weights).flatten()

    # Trier les indices des chansons par similarité décroissante
    similar_indices = similarities.argsort()[::-1]
    # Exclure les chansons de la playlist elle-même et prendre les num_recommendation plus proches
    similar_indices = [idx for idx in similar_indices if idx not in playlist_tracks.index][:num_recommendation]

    # Récupérer les informations des chansons similaires
    similar_tracks = data_songs.iloc[similar_indices][['track_name', 'track_artist']]
    similar_tracks['similarity'] = similarities[similar_indices]

    return similar_tracks.reset_index(drop=True)

# Exemple d'utilisation
recommend_similar_tracks_playlist("Dance Pop: Japan", num_recommendation=3)

L'algorithme nous propose les morceaux suivants :
* Bailame Despacio - Nacho, Yandel, Bad Bunny
* Notorious Thugs - The Notorious B.I.G., 2Pac, Diddy
* Wasted on You - Louis Futon

Après écoute, ces morceaux correspondent bien à l'esprit de la playlist Dance Pop: Japan. Ils sont dansants, joyeux et dynamiques, ce qui correspond au profil musical de la playlist.

# Conclusion

Ce projet d'analyse de données musicales a permis d'explorer en profondeur un dataset riche de plus de 30 000 morceaux issus de playlists Spotify, en appliquant diverses techniques d'analyse multivariée et d'apprentissage non supervisé.

### Apports méthodologiques

**Maîtrise d'un pipeline complet d'analyse**  
Ce projet nous a permis de développer une expertise transversale en :
- **Préparation et nettoyage** de données complexes
- **Analyse exploratoire** approfondie avec visualisations avancées
- **Application coordonnée** de multiples techniques d'analyse multivariée
- **Validation et comparaison** rigoureuse des méthodes de clustering

**Expertise en techniques d'analyse multivariée**  
Nous avons acquis une maîtrise pratique de :
- L'**Analyse en Composantes Principales (ACP)** pour la réduction de dimensionnalité
- L'**Analyse des Correspondances Multiples (ACM)** pour les variables qualitatives
- La **Factorisation Matricielle Non-Négative (NMF)** pour l'extraction de profils latents
- Les **méthodes de clustering** (K-Means, GMM, CAH) avec optimisation des hyperparamètres

### Découvertes scientifiques

**Structures cachées dans les données musicales**  
L'analyse a révélé des **patterns significatifs** :
- **6 profils musicaux latents** distincts identifiés par NMF
- **Correspondances fortes** entre caractéristiques audio et genres musicaux
- **Segmentation naturelle** des morceaux en groupes homogènes

**Validation de la cohérence des résultats**  
La convergence des différentes méthodes confirme la **robustesse** de nos analyses :
- Les clusters correspondent aux genres attendus
- Les profils NMF sont musicalement interprétables
- La MCA révèle des associations logiques entre variables

### Compétences techniques développées

**Programmation avancée en Python**  
- Manipulation de **pandas** pour les données structurées
- Visualisation sophistiquée avec **matplotlib/seaborn**
- Implémentation d'algorithmes **scikit-learn** 

### Applications concrètes

Ce projet démontre notre capacité à :
- **Transformer** des données brutes en insights exploitables
- **Développer** des systèmes de recommandation fonctionnels

### Perspectives professionnelles

Cette expérience nous prépare à :
- **Analyser** tout type de données multidimensionnelles
- **Conseiller** sur le choix de méthodes analytiques appropriées
- **Communiquer** des résultats techniques

Ce projet illustre parfaitement comment l'analyse de données peut révéler des structures cachées dans des domaines complexes, tout en développant une expertise technique solide et une approche méthodologique rigoureuse.